<a href="https://colab.research.google.com/github/joshheyer/CONAFOR/blob/main/3-LandTrendr_and_CCDCV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Copyright 2023 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
#
# Developed for Google LLC by RedCastle Resources Inc
# https://www.redcastleresources.com/

# <img width=50px  src = 'https://apps.fs.usda.gov/lcms-viewer/images/lcms-icon.png'>  Laboratorio 3: LandTrendr y CCDC

<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/redcastle-resources/CONAFOR/blob/main/3-LandTrendr_and_CCDC_espanol.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> Run in Colab
    </a>
  </td>
  <td>
    <a href="https://github.com/redcastle-resources/CONAFOR/blob/main/3-LandTrendr_and_CCDC_espanol.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      View on GitHub
    </a>
  </td>
</table>
<br/><br/><br/>

## 3.0: Descripción General e Introducción

### LandTrendr
LandTrendr, abreviatura de detección de tendencias en perturbaciones y recuperación basada en Landsat, es un algoritmo de segmentación temporal que se utiliza para la detección de cambios y el suavizado de series temporales (segmentación temporal). Puede utilizar las salidas de LandTrendr para la detección de cambios independientes o como entradas para modelos de modelado como los utilizados por LCMS.

#### Más información sobre LandTrendr:
- [LandTrendr Guide](https://emapr.github.io/LT-GEE/)
- [LandTrendr Original Publication](https://www.sciencedirect.com/science/article/abs/pii/S0034425710002245)
- [LandTrendr GEE Publication](https://www.mdpi.com/2072-4292/10/5/691)

### CCDC
CCDC significa Detección y Clasificación de Cambios Continuos. CCDC tiene una definición fundamentalmente diferente de lo que es "cambio" de LandTrendr. LandTrendr define el cambio como un cambio en la dirección lineal de la serie temporal como se muestra con un modelo de regresión lineal, mientras que CCDC define el cambio como un cambio en la estacionalidad (fenología) como se muestra usando una regresión armónica (regresión lineal sobre muchas formas de onda diferentes).

Como resultado, en general, la descripción del cambio que hace LandTrendr se alinea con muchos tipos de cambios relacionados con los bosques, como incendios, insectos y enfermedades, etc. Si bien estos tipos de cambios a menudo cambian la dirección de la trayectoria abruptamente, no siempre cambian la estacionalidad. patrones de manera abrupta.

CCDC puede detectar mejor cambios que afectan la fenología y que LandTrendr puede pasar por alto. Esto puede resultar útil en aplicaciones urbanas, agrícolas y de pastizales.

#### Más información sobre CCDC:
- [Zhu and Woodock 2014: Original CCDC Publication](https://www.sciencedirect.com/science/article/abs/pii/S0034425714000248)
- [GEE CCDC Documentation](https://developers.google.com/earth-engine/apidocs/ee-algorithms-temporalsegmentation-ccdc)
- [Arevalo et al. 2020: A Suite of Tools for Continuous Land Change Monitoring in Google Earth Engine](https://www.frontiersin.org/articles/10.3389/fclim.2020.576740/full)
- [Cloud-Based Remote Sensing with Google Earth Engine](https://www.eefabook.org/) Chapter 4.6: [Interpreting Time Series with CCDC](https://docs.google.com/document/d/11oo1TuvXyEvReoYLDeTcoh16rEN90I8Bf5qp0KxVu7U/edit#heading=h.s893qtc8bydw)

**Este cuaderno utiliza compuestos Landsat y Sentinel-2, como los que creó en el cuaderno anterior, como entradas para la operación de suavizado espectral de LandTrendr..**

### 3.0.1: Objetivos

En este tutorial, aprenderá cómo crear y manipular resultados de LandTrendr.

**Este tutorial utiliza los siguientes servicios de Google Cloud:**

- Google Earth Engine

**Los pasos realizados incluyen:**

- Crear resultados de LandTrendr
- Manipule objetos de imagen de matriz EE para obtener datos significativos de las salidas de LandTrendr para la detección y clasificación de cambios.
- Crear una cuadrícula de malla de mosaicos sobre un área de estudio
- Cree salidas CCDC sobre la cuadrícula de malla
- Manipule objetos de imagen de matriz EE para obtener datos significativos de las salidas CCDC para la detección y clasificación de cambios.

**Los objetivos de aprendizaje incluyen:**
- Los usuarios comprenderán el propósito de la segmentación temporal implementada en LandTrendr..
- Los usuarios comprenderán los parámetros clave del algoritmo LandTrendr.
- Los usuarios podrán generar y manipular resultados de matriz desde LandTrendr..
- Los usuarios comprenderán el propósito de la segmentación temporal tal como se implementa en CCDC.
- Los usuarios comprenderán los parámetros clave del algoritmo CCDC.
- Los usuarios podrán generar y manipular salidas de matriz desde CCDC.


### 3.0.2: Antes de que empieces

#### Si está trabajando en Workbench: establezca su URL actual en `workbench_url`
Esto le da al Visor de mapas una URL en la que alojar el visor que generaremos.
* Esto estará en su URL/barra de búsqueda en la parte superior de la ventana del navegador en la que se encuentra actualmente
* Se verá algo así como `https://1234567890122-dot-us-west3.notebooks.googleusercontent.com/` (Vea la imagen a continuación)

![workspace url](https://github.com/redcastle-resources/lcms-training/blob/main/img/workspace-url.png?raw=1)

#### Opcional: establezca una carpeta para usar en todas las exportaciones en `export_path_root`
**Si accedió a esta práctica de laboratorio a través de Google Qwiklabs, no cambie `export_path_root`.**
* Esta carpeta debe ser una carpeta de activos en un proyecto GEE existente.
* De forma predeterminada, esta carpeta es la misma que la 'carpeta predefinida': un directorio donde las salidas ya se han creado.
* Si desea crear sus propios resultados, especifique una ruta diferente para `export_path_root`, pero deja el `pre_baked_path_root` como era. De esta manera, las salidas precocidas se pueden mostrar al final, en lugar de esperar a que terminen todas las exportaciones.
* será algo así como `projects/projectID/assets/someFolder`
* No es necesario que esta carpeta ya exista. Si no existe, se creará

In [1]:
workbench_url = 'https://1307eb830a12e633-dot-us-central1.notebooks.googleusercontent.com/lab/tree/lcms-training/3-LandTrendr_and_CCDC.ipynb'
pre_baked_path_root  = 'projects/ee-jheyer2325/assets'
export_path_root = pre_baked_path_root
print('Hecho')

Done


#### Instalación
Primero, instale los paquetes de Python necesarios. Descomente la primera línea para actualizar geeViz si es necesario.

Tenga en cuenta que para este módulo, también estamos importando el `geeViz.changeDetectionLib as changeDetectionLib`. Usaremos esta biblioteca más adelante para implementar las funciones de LandTrendr.

In [2]:
# Importaciones de módulos
#!python -m pip install geeViz --upgrade
try:
    import geeViz.getImagesLib as getImagesLib
except:
    !python -m pip install geeViz
    import geeViz.getImagesLib as getImagesLib

import geeViz.changeDetectionLib as changeDetectionLib
import geeViz.assetManagerLib as aml
import geeViz.taskManagerLib as tml
import geeViz.gee2Pandas as g2p

import inspect,os

ee = getImagesLib.ee
Map = getImagesLib.Map

print('Hecho')

Initializing GEE


*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_doiqkQG3NJ1t8IS?source=API


Successfully initialized
geeViz package folder: c:\Users\joshuaheyer\AppData\Local\Programs\Python\Python311\Lib\site-packages\geeViz
PyTables is not installed. No support for HDF output.
SQLalchemy is not installed. No support for SQL output.
Done


#### Configura tu entorno de trabajo

Especifique una ruta de colección de imágenes a la que se exportaron los compuestos. Además, cree una colección de imágenes en blanco donde se exportarán sus resultados de LandTrendr.

Actualmente, cuando se ejecuta en Colab o Workbench, geeView utiliza un proyecto diferente para autenticarse, por lo que es posible que deba hacer público su recurso para verlo desde Colab.

**Atención!!**

* **Se proporciona acceso de solo lectura a todos los usuarios de GEE autenticados para los resultados precocidos**
* **Si está utilizando la ubicación de salida precocida (`export_path_root = pre_baked_path_root`), Verá errores para cualquier operación que intente escribir, eliminar o cambiar los permisos de acceso a cualquier recurso en la ubicación de salida predefinida.**
* **Esto es lo esperado y no le impedirá ejecutar correctamente este cuaderno. Ignore estos mensajes de error si aparecen.**


In [3]:
# Crear carpeta y una colección y hacerla pública

export_composite_collection = f'{export_path_root}/lcms-training_module-2_composites'
export_landTrendr_collection = f'{export_path_root}/lcms-training_module-3_landTrendr'
export_ccdc_collection = f'{export_path_root}/lcms-training_module-3_CCDC'

aml.create_asset(export_landTrendr_collection,asset_type = ee.data.ASSET_TYPE_IMAGE_COLL)
aml.create_asset(export_ccdc_collection,asset_type = ee.data.ASSET_TYPE_IMAGE_COLL)

# Actualmente, geeView dentro de Colab utiliza un proyecto diferente para autenticarse, por lo que es posible que deba hacer público su activo para verlo desde Colab
aml.updateACL(export_landTrendr_collection,writers = [],all_users_can_read = True,readers = [])

print('Hecho')

Asset projects/ee-jheyer2325/assets/lcms-training_module-3_landTrendr already exists
Asset projects/ee-jheyer2325/assets/lcms-training_module-3_CCDC already exists
Updating permissions for:  projects/ee-jheyer2325/assets/lcms-training_module-3_landTrendr
Done


#### Configurar el mapa

Ejecute el bloque de código a continuación para configurar el mapa. Si observa que el mapa todavía muestra los resultados de una práctica de laboratorio anterior, es posible que deba volver a ejecutar este bloque de código para restablecer el puerto utilizado para la URL del proxy.

In [4]:
# Configurar el mapa

Map.clearMap()
Map.port = 1233 # restablecer el puerto si es necesario
Map.proxy_url = workbench_url

## 3.2: Detección de cambios con LandTrendr

LandTrendr es un algoritmo de segmentación temporal. LandTrendr funciona tomando una serie de tiempo de valores anuales y ajustando múltiples modelos de regresión lineal para dividir recursivamente la serie de tiempo en segmentos que representan períodos de tiempo con tendencias lineales similares. Los segmentos resultantes se pueden utilizar para describir procesos de cambio en el paisaje.

![landtrendr1](https://emapr.github.io/LT-GEE/imgs/segmentation.png)

Al tener la duración, magnitud, pendiente, etc. de cada segmento, puede detectar cambios más fácilmente con conjuntos de reglas simples..

![landtrendr2](https://emapr.github.io/LT-GEE/imgs/segment_attributes.png)

Por ejemplo, una caída breve y pronunciada probablemente se convierta en un cambio rápido, como una cosecha o un incendio. Es probable que una disminución prolongada y superficial sea un cambio más lento, debido a insectos, enfermedades o sequía. LandTrendr también puede rastrear períodos de recuperación, como una pendiente larga y poco profunda después de una cosecha o un incendio.

**Para obtener más información sobre LandTrendr, lea [Chapter 4.5: Interpreting Annual Time Series with LandTrendr](https://docs.google.com/document/d/11oo1TuvXyEvReoYLDeTcoh16rEN90I8Bf5qp0KxVu7U/edit#heading=h.a480u2bjy8ur) en el [Cloud-Based Remote Sensing with Google Earth Engine ebook](https://www.eefabook.org/).**


### 3.2.1: Bandas y parámetros de LandTrendr

Mire las bandas incluidas en la colección **actual** que LCMS usa para LandTrendr en Puerto Rico. Tenga en cuenta que la ruta al activo hace referencia al proyecto LCMS Google Cloud. Cargue la colección de imágenes e imprima los nombres de las bandas.

In [5]:
lcms_actual_lt_collection = ee.ImageCollection('projects/lcms-292214/assets/R8/PR_USVI/Base-Learners/LandTrendr-Collection-1984-2020')
print('Todas las bandas que LCMS utiliza para LandTrendr:',lcms_actual_lt_collection.aggregate_histogram('band').keys().getInfo())

All bands LCMS uses for LandTrendr: ['NBR', 'NDMI', 'NDSI', 'NDVI', 'blue', 'brightness', 'green', 'greenness', 'nir', 'red', 'swir1', 'swir2', 'tcAngleBG', 'wetness']


#### Compuestos
Para este laboratorio, usted
    a) cargue los compuestos que se calcularon en la práctica de laboratorio anterior para
    b) calcular los resultados de LandTrendr ajustados que indican el momento, la duración y la magnitud del cambio, entre otros resultados.

Ejecute el bloque de código a continuación. Este bloque de código:
* Inspecciona los compuestos que estás trayendo
* Obtenga parámetros de los compuestos que utilizará como entradas para el algoritmo LandTrendr
* Calcular índices adicionales que utilizará en LandTrendr

In [6]:
# Trae compuestos y extrae información de ellos
composites = ee.ImageCollection(export_composite_collection)

props = composites.first().toDictionary().getInfo()

startYear = props['startYear']
endYear = props['endYear']

startJulian = props['startJulian']
endJulian = props['endJulian']

proj = composites.first().projection().getInfo()

# Saca los crs
# Dependiendo de si se utiliza un formato wkt o epsg, se almacenará con una clave diferente
if 'crs' not in proj.keys():
    crs = proj['wkt']
else:
    crs = proj['crs']

transform = proj['transform']
scale = None

studyArea = composites.first().geometry()

# Descomprima compuestos dividiéndolos por 10000 para bandas ópticas y agregue índices
composites = composites.select(['blue','green','red','nir','swir1','swir2']).map(lambda img: img.divide(10000).float().copyProperties(img,['system:time_start']))
composites = composites.map(getImagesLib.simpleAddIndices)\
                      .map(getImagesLib.getTasseledCap)\
                      .map(getImagesLib.simpleAddTCAngles)


print('Hecho')

Done


#### Inspeccionar
Ahora, inspecciona los compuestos. Ejecute el bloque de código siguiente para agregar los compuestos al mapa. Es posible que deba hacer clic en el botón junto a "Compuestos" para ver las capas en el mapa. Haga doble clic en el mapa para consultar las salidas. Deberías ver una serie temporal de valores para cada uno de los índices.

In [7]:
# Ejecutar mapa
Map.clearMap()
Map.addTimeLapse(composites,getImagesLib.vizParamsFalse,'Composites')

Map.centerObject(studyArea,9)
Map.turnOnInspector()
Map.view()

Adding layer: Composites
Starting webmap
Using default refresh token for geeView: C:\Users\joshuaheyer/.config/earthengine/credentials
Starting local web server at: http://localhost:1233/geeView/
HTTP server command: "c:\Users\joshuaheyer\AppData\Local\Programs\Python\Python311\python.exe" -m http.server  1233
Done
cwd c:\Users\joshuaheyer\Scripts\CONAFOR


### 3.2.3: Bandas de entrada e índices de bandas

Podemos usar cualquiera o todas las bandas en los compuestos para LandTrendr, pero generalmente las bandas o índices de bandas que usan las porciones NIR y SWIR del espectro electromagnético (sensibles a la humedad y la clorofila fotosintética) son las más útiles. Para comenzar, ejecutaremos LandTrendr solo en el índice de banda: NBR.

#### LandTrendr parámetros
Los parámetros de entrada para el algoritmo LandTrendr se describen a continuación.

|Argumento               |Tipo                    | Detalles                                                                                                                                                       |
|------------------------|-------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| timeSeries             | ImageCollection         | Series temporales anuales de las que extraer puntos de interrupción. La primera banda se utiliza para encontrar puntos de interrupción y todas las bandas posteriores se ajustan utilizando esos puntos de interrupción.     |
| maxSegments            | Integer                 | Número máximo de segmentos que se ajustarán en la serie temporal.                                                                                                       |
| spikeThreshold         | Float, default: 0.9     | Umbral para humedecer los picos (1,0 significa sin humedecimiento).                                                                                                      |
| vertexCountOvershoot   | Integer, default: 3     | TEl modelo inicial puede sobrepasar los maxSegments + 1 vértices en esta cantidad. Más tarde, se reducirá a maxSegments + 1.                                    |
| preventOneYearRecovery | Boolean, default: False | Prevenir segmentos que representan recuperaciones de un año.                                                                                                              |
| recoveryThreshold      | Float, default: 0.25    | Si un segmento tiene una tasa de recuperación superior a 1/umbral de recuperación (en años), entonces el segmento no está permitido.                                                      |
| pvalThreshold          | Float, default: 0.1     | Si el valor p del modelo ajustado excede este umbral, entonces el modelo actual se descarta y se ajusta otro usando el optimizador de Levenberg-Marquardt. |
| bestModelProportion    | Float, default: 0.75    | Permite elegir modelos con más vértices si su valor p no es mayor que (2 - bestModelProportion) veces el valor p del mejor modelo.                     |
| minObservationsNeeded  | Integer, default: 6     | Observaciones mínimas necesarias para realizar el ajuste de salida.                                                                                                                |

##### Valores paramétricos
LCMS generalmente utiliza los parámetros predeterminados para la implementación GEE de LandTrendr. Consulte los valores de los parámetros en la columna **GEE** a continuación. La siguiente tabla es una reproducción de [Kennedy et al. 2018](https://www.mdpi.com/2072-4292/10/5/691), que desarrolló la implementación GEE de LandTrendr a partir de la versión IDL original.

|       Parámetro       |  IDL |   GEE   |                                                                                                        Comments                                                                                                       |
|:---------------------:|:----:|:-------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------:|
|      maxSegments      |   6  |    6    |                                                                                                                                                                                                                       |
|     spikeThreshold    |  0.9 |   0.9   | Renombrado de “desawtooth val”                                                                                                                                                                                         |
|  vertexCountOvershoot |   3  |    3    |                                                                                                                                                                                                                       |
|   recoveryThreshold   | 0.25 |   0.25  |                                                                                                                                                                                                                       |
|     pvalThreshold     | 0.05 |   0.05  |                                                                                                                                                                                                                       |
|  bestModelProportion  | 0.75 |   0.75  |                                                                                                                                                                                                                       |
| minObservationsNeeded |   6  |    6    | Renombrado de “minneeded”                                                                                                                                                                                              |
|     Background_val    |   0  |    NA   | GEE utiliza una lógica de máscara para evitar valores perdidos causados ​​por nubes, sombras e imágenes faltantes.                                                                                                                         |
|        Divisor        |  −1  |    NA   | EGarantiza que la alteración de la pérdida de vegetación produzca un cambio negativo en el valor cuando se utiliza NBR como métrica espectral. En GEE, esto debe manejarse fuera del algoritmo de segmentación.                               |
|       Kernelsize      |   1  | Caído | Originalmente utilizado junto con skipfactor para ahorrar carga computacional; ya no es necesario.                                                                                                                           |
|       Skipfactor      |   1  | Caído |                                                                                                                                                                                                                       |
|    Distweightfactor   |   2  | Caído | Inadvertidamente cableado en el código IDL, este parámetro estaba cableado en el código GEE al valor de 2.                                                                                                              |
|     Fix_doy_effect    |   1  | Caído | Aunque la corrección de las tendencias del día del año se consideró teóricamente útil en la implementación original de LT, en la práctica se ha descubierto que distorsiona los valores de las series temporales cuando se producen cambios y, por lo tanto, se eliminó. |

**Ejecute el bloque de código a continuación para configurar los parámetros.**

In [8]:
#Definir los parámetros de landtrendr
run_params = {
  'maxSegments':            6,
  'spikeThreshold':         0.9,
  'vertexCountOvershoot':   3,
  'preventOneYearRecovery': False,
  'recoveryThreshold':      0.25,
  'pvalThreshold':          0.05,
  'bestModelProportion':    0.75,
  'minObservationsNeeded':  6
}

print('Hecho')

Done


### 3.2.4: Ejecute LandTrendr - índice único
Comenzará ejecutando LandTrendr en un único índice para familiarizarse con el proceso y los resultados.

Ejecutará LandTrendr usando [Normalized Burn Ratio](https://www.usgs.gov/landsat-missions/landsat-normalized-burn-ratio) (NBR).  NBR es una relación normalizada entre las bandas NIR y SWIR y es sensible a cambios en la humedad y la cubierta vegetal. Esto le permitirá inspeccionar cambios en el paisaje sin requerir el tiempo de cálculo necesario para todas las bandas e índices de bandas en nuestro compuesto.



In [9]:
# Limpiar el mapa
Map.clearMap()

# Configurar LandTrendr
test_band = 'NBR'
run_params['timeSeries'] = composites.select([test_band])
raw_LT = ee.Algorithms.TemporalSegmentation.LandTrendr(**run_params)

# Añadir al mapa
Map.addLayer(raw_LT,{},'LT Raw {}'.format(test_band),True)

Map.turnOnInspector()
Map.view()

Adding layer: LT Raw NBR
Starting webmap
Using default refresh token for geeView: C:\Users\joshuaheyer/.config/earthengine/credentials
Local web server at: http://localhost:1233/geeView/ already serving.
cwd c:\Users\joshuaheyer\Scripts\CONAFOR


Haga doble clic en la salida en el mapa para consultarla..

Tenga en cuenta que este resultado no es útil de inmediato para la detección de cambios o para suavizar una serie temporal de compuestos. La salida predeterminada está en formato de matriz y debe procesarse más para que podamos aplicar una simbología significativa en el mapa.


#### Utilice LandTrendr para la detección de cambios
Usando los resultados que acaba de generar, ahora seguirá cada paso para tomar el resultado sin procesar de LandTrendr y crear un resultado básico de detección de cambios.

En cada paso, verá los resultados en un solo píxel para inspeccionar los cambios en la matriz de salidas que está manipulando.

#### Inspeccionar resultados sin procesar de LandTrendr

Ejecute el siguiente bloque de código para inspeccionar las salidas sin procesar de LandTrendr.

In [10]:
# Proporcione una ubicación de ejemplo
pt = ee.Geometry.Point([-93.63546,17.78971])

# Primero, seleccione la banda de salida de la matriz de imágenes LandTrendr
lt_array = raw_LT.select(['LandTrendr'])

# Salida de pantalla
display(g2p.imageArrayPixelToDataFrame(lt_array, pt,None,crs,transform, 'Raw LandTrendr Output - Single Pixel',\
                                       ['Years','Raw Input Values','Fitted Output Values','Vertex/non-vertex']))

Matplotlib is building the font cache; this may take a moment.


,0,1,2,3,4,5,6
Years,1985.000000,1987.000000,1989.000000,1992.000000,1993.000000,1994.000000,1995.000000
Raw Input Values,0.411179,0.410844,0.435623,0.530902,0.554928,0.514331,-0.023077
Fitted Output Values,0.404961,0.404961,0.404961,0.404961,0.404961,0.404961,0.404961
Vertex/non-vertex,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000


Este es el resultado bruto del algoritmo LandTrendr. La matriz tiene 2 dimensiones por píxel.

Las filas corresponden a:
- Años
- Valores espectrales de entrada sin procesar
- Valores de salida ajustados de LandTrendr
- Si ese año representa o no un vértice

A partir de este resultado, puede comenzar a comprender el formato de los resultados, por qué no son interpretables inmediatamente en un mapa y cómo podría comenzar a manipularlos en formatos más significativos..

## 3.3: Procese las salidas de LandTrendr y ejecútelas en todas las bandas

### 3.3.1: Manipulación de matrices: extraer los vértices

El primer paso es extraer los vértices de la matriz. Solo necesitamos los vértices de un píxel para realizar un seguimiento del cambio; no necesitamos los valores de los años intermedios..

Ejecute el bloque de código siguiente para extraer los vértices. Corta la matriz para extraer la fila que indica los vértices y los usa como una máscara para enmascarar los valores que no son vértices en toda la matriz.

In [11]:
# Cortar la matriz para extraer la fila que indica los vértices
vertices = lt_array.arraySlice(0,3,4)
display(g2p.imageArrayPixelToDataFrame(vertices, pt, None,crs,transform,'Vertex mask row'))

# Utilice la fila de vértices como máscara para extraer los valores en los vértices
lt_array = lt_array.arrayMask(vertices)
display(g2p.imageArrayPixelToDataFrame(lt_array, pt, None,crs,transform,'Raw LandTrendr - Only Vertex Columns',\
                                                 ['Years','Raw Input Values','Fitted Output Values','Vertex/non-vertex']))

,0,1,2,3,4,5,6
0,1,0,0,0,0,0,1


,0,1
Years,1985.000000,1995.000000
Raw Input Values,0.411179,-0.023077
Fitted Output Values,0.404961,0.404961
Vertex/non-vertex,1.000000,1.000000


#### Calcular la diferencia entre los valores de vértices ajustados
Para realizar la detección de cambios, necesitará obtener la diferencia entre los valores de vértice ajustados.

Esto se hace dividiendo la matriz en un desplazamiento, para poder restar valores adyacentes.

Ejecute el bloque de código siguiente para calcular la diferencia entre los valores de vértice ajustados.

In [12]:
# Para realizar la detección de cambios, necesitaremos obtener la diferencia entre los valores de los vértices ajustados.
# Hacemos esto cortando el primero al penúltimo y luego el penúltimo al último y restándolos.
left = lt_array.arraySlice(1,0,-1)
right = lt_array.arraySlice(1,1,None)
diff  = right.subtract(left)

display(g2p.imageArrayPixelToDataFrame(left, pt, None,crs,transform,'Left Slice',\
                                       ['Years','Raw Input Values','Fitted Output Values','Vertex/non-vertex']))
display(g2p.imageArrayPixelToDataFrame(right, pt,None,crs,transform, \
                                       'Right Slice',['Years','Raw Input Values','Fitted Output Values','Vertex/non-vertex']))
display(g2p.imageArrayPixelToDataFrame(diff, pt, None,crs,transform,\
                                       'Right Minus Left',['Years','Raw Input Values','Fitted Output Values','Vertex/non-vertex']))

,0
Years,1985.000000
Raw Input Values,0.411179
Fitted Output Values,0.404961
Vertex/non-vertex,1.000000


,0
Years,1995.000000
Raw Input Values,-0.023077
Fitted Output Values,0.404961
Vertex/non-vertex,1.000000


,0
Years,10.000000
Raw Input Values,-0.434256
Fitted Output Values,0.000000
Vertex/non-vertex,0.000000


#### Combina valores de diferencia con años.
Luego, divide los años de la derecha y la diferencia de valores de vértice ajustados y los combina.

Ejecute el bloque de código siguiente para crear una matriz que tenga un valor para cada año de vértice y magnitud de diferencia.

In [13]:
# Cortar los años de la derecha y los valores de diferencia
years = right.arraySlice(0,0,1)
mag = diff.arraySlice(0,2,3)
display(g2p.imageArrayPixelToDataFrame(years, pt, None,crs,transform,'Years'))
display(g2p.imageArrayPixelToDataFrame(mag, pt, None,crs,transform,'Magnitude'))

# Combinar
forSorting = years.arrayCat(mag,0)
display(g2p.imageArrayPixelToDataFrame(forSorting, pt, None,crs,transform,'Year + Magnitude Array'))

,0
0,1995


,0
0,0


,0
0,1995
1,0


#### Ordenar matriz según el cambio de interés

Luego podemos ordenar esta matriz para mostrar el cambio que más nos interesa. Por ejemplo, podemos extraer la pérdida de mayor magnitud, la pérdida más reciente, etc.

En el siguiente ejemplo, la fila de clasificación será la magnitud. Por lo tanto, la salida será la pérdida de gravedad más alta.


In [14]:
# Ordenar por magnitud
sorted = forSorting.arraySort(forSorting.arraySlice(0,1,2))
display(g2p.imageArrayPixelToDataFrame(sorted, pt, None,crs,transform,'Array Sorted by the Second Row (magnitude of loss)'))

# Separe el año y la magnitud de la pérdida de mayor magnitud
highest_mag_change_array = sorted.arraySlice(1,0,1)
display(g2p.imageArrayPixelToDataFrame(highest_mag_change_array, pt, None,crs,transform,'Highest Mag Loss (year and magnitude)'))

,0
0,1995
1,0


,0
0,1995
1,0


#### Convertir matrices en imágenes

El último paso es convertir la salida de la matriz en una imagen. Has visto cómo funciona la manipulación de matrices en un píxel. Ahora puede "aplanar" la matriz bidimensional en una imagen unidimensional.

La aplicación de un umbral de cambio le permite determinar qué gravedad del cambio se marca como pérdida. Todo cambio que sea inferior a esta magnitud quedará oculto en el mapa.

Ejecute el bloque de código siguiente para aplanar la matriz en una imagen y agregarla al mapa.

In [15]:
# Primero, elija un umbral de pérdida
# Cualquier cambio más negativo que este valor se marcará como pérdida
change_threshold = -0.15

# Convierta la imagen de matriz ordenada en una imagen de 2 bandas
highest_mag_change = highest_mag_change_array.arrayProject([0]).arrayFlatten([['yr','mag']])

# Ocultar cualquier pérdida
highest_mag_change = highest_mag_change.updateMask(highest_mag_change.select(['mag']).lte(change_threshold))

# Extraiga la paleta de magnitud de pérdida y cambie el orden de los colores
lossMagPalette = changeDetectionLib.lossMagPalette.split(',')
lossMagPalette.reverse()

# Configurar mapa
Map.clearMap()
Map.addLayer(forSorting,{},'Array for Sorting',False)
Map.addLayer(sorted,{},'Sorted Array',False)
Map.addLayer(highest_mag_change.select(['mag']),{'min':-0.8,'max':-0.15,'palette':lossMagPalette},'Loss Magnitude')
Map.addLayer(highest_mag_change.select(['yr']),{'min':startYear,'max':endYear,'palette':changeDetectionLib.lossYearPalette},'Loss Year')
Map.turnOnInspector()
Map.view()

Adding layer: Array for Sorting
Adding layer: Sorted Array
Adding layer: Loss Magnitude
Adding layer: Loss Year
Starting webmap
Using default refresh token for geeView: C:\Users\joshuaheyer/.config/earthengine/credentials
Local web server at: http://localhost:1233/geeView/ already serving.
cwd c:\Users\joshuaheyer\Scripts\CONAFOR


#### Inspeccionar la salida
Ver la salida en el mapa. Desactive y active las capas para ver los rásteres Año de pérdida y Magnitud de pérdida. Haga doble clic en un píxel para consultarlo y vea los valores de la matriz original para comprender cómo se corresponden con la salida final.

### 3.3.2: Ejecute LandTrendr - en todas las bandas

Ahora que comprende los parámetros y resultados de LandTrendr, el siguiente paso es ejecutar LandTrendr en todas las bandas. Este es un resultado más realista. Después de un procesamiento adicional, utilizará este resultado con el modelo LCMS en los Módulos 4 y 5.

Tenga en cuenta que el código siguiente es el mismo que utilizó para ejecutar LandTrendr en una sola banda. Pero aquí, está aplicando LandTrendr sobre cada banda y no muestra los resultados de la manipulación de la matriz en cada paso.

Ejecute el bloque de código siguiente para calcular y exportar matrices de salida de LandTrendr para cada una de las bandas enumeradas en `bandNames`.

In [16]:
# Ejecutaremos LandTrendr para cada banda
# Borre el mapa en caso de que se haya poblado con capas/comandos anteriormente
Map.clearMap()

# Podemos usar cualquiera o todas las bandas, pero generalmente las bandas que usan nir y swir son las más útiles
bandNames = ['red','nir','swir1','swir2','NBR','NDVI','brightness','greenness','wetness']

# Ejecute LANDTRENDR
for bandName in bandNames:

    # Seleccione la banda y ejecute LandTrendr
    run_params['timeSeries'] = composites.select([bandName])
    rawLT = ee.Algorithms.TemporalSegmentation.LandTrendr(**run_params)

    Map.addLayer(rawLT,{},'LT Raw {}'.format(bandName),False)

    # Observe que la salida sin procesar de LandTrendr está en el formato de matriz de imágenes de GEE
    # Necesitaremos manipular un poco la salida sin procesar para ahorrar espacio de almacenamiento

    # Enmascare los valores que no sean vértices para utilizar menos espacio de almacenamiento
    ltArray = rawLT.select(['LandTrendr'])
    rmse = rawLT.select(['rmse'])
    vertices = ltArray.arraySlice(0,3,4)
    ltArray = ltArray.arrayMask(vertices)

    # Enmascare todos menos los valores ajustados de año y vértice (elimine las filas sin formato y de vértice)
    ltArray = ltArray.arrayMask(ee.Image(ee.Array([[1],[0],[1],[0]])))
    rawLTForExport=ltArray.addBands(rmse)
    Map.addLayer(rawLTForExport,{},'LT Vertex Values Only {}'.format(bandName),False)

    # Muestre cómo los valores comprimidos solo de vértice se pueden descomprimir más adelante
    decompressedC = changeDetectionLib.simpleLTFit(ltArray,startYear,endYear,bandName,True,run_params['maxSegments'])
    Map.addLayer(decompressedC,{'bands':'{}_LT_fitted'.format(bandName),'min':0.2,'max':0.8},'Decompressed LT Output {}'.format(bandName),False)

    # Unir los valores brutos y ajustados
    fitted = decompressedC.select(['{}_LT_fitted'.format(bandName)])
    ltJoined = getImagesLib.joinCollections(composites.select([bandName]),fitted)
    Map.addLayer(ltJoined,{'bands':'{}_LT_fitted'.format(bandName),'min':0.2,'max':1,'palette':'D80,080'},'Raw and LT Fitted {}'.format(bandName),True)

    # Exportar imagen de matriz LT
    # Establece algunas propiedades que se usarán más adelante
    rawLTForExport = rawLTForExport.set({'startYear':startYear,
                                          'endYear':endYear,
                                          'startJulian':startJulian,
                                          'endJulian':endJulian,
                                          'band':bandName})
    rawLTForExport =rawLTForExport.set(run_params)
    exportName = 'LT_Raw_{}_yrs{}-{}_jds{}-{}'.format(bandName,startYear,endYear,startJulian,endJulian)
    exportPath = export_landTrendr_collection + '/'+ exportName
    # Exportar producción
    getImagesLib.exportToAssetWrapper(rawLTForExport,exportName,exportPath,{'.default':'sample'},studyArea,scale,crs,transform,overwrite=False)

Map.turnOnInspector()
Map.addLayer(studyArea, {'strokeColor': '0000FF'}, "Study Area", False)
Map.view()

Adding layer: LT Raw red
Adding layer: LT Vertex Values Only red
Adding layer: Decompressed LT Output red
Adding layer: Raw and LT Fitted red
Exporting: LT_Raw_red_yrs1984-2022_jds152-151
Adding layer: LT Raw nir
Adding layer: LT Vertex Values Only nir
Adding layer: Decompressed LT Output nir
Adding layer: Raw and LT Fitted nir
Exporting: LT_Raw_nir_yrs1984-2022_jds152-151
Adding layer: LT Raw swir1
Adding layer: LT Vertex Values Only swir1
Adding layer: Decompressed LT Output swir1
Adding layer: Raw and LT Fitted swir1
Exporting: LT_Raw_swir1_yrs1984-2022_jds152-151
Adding layer: LT Raw swir2
Adding layer: LT Vertex Values Only swir2
Adding layer: Decompressed LT Output swir2
Adding layer: Raw and LT Fitted swir2
Exporting: LT_Raw_swir2_yrs1984-2022_jds152-151
Adding layer: LT Raw NBR
Adding layer: LT Vertex Values Only NBR
Adding layer: Decompressed LT Output NBR
Adding layer: Raw and LT Fitted NBR
Exporting: LT_Raw_NBR_yrs1984-2022_jds152-151
Adding layer: LT Raw NDVI
Adding layer: 

Si desea realizar un seguimiento del estado de las tareas de exportación, utilice el siguiente código.

In [ ]:
# Puede realizar un seguimiento de las tareas aquí o en https://code.earthengine.google.com/tasks
# Si desea realizar un seguimiento de las tareas, utilice esto:
# tml.trackTasks2()

# Si desea cancelar todas las tareas en ejecución, puede utilizar esta función
# tml.batchCancel()

# Si desea vaciar la colección de todas las imágenes
# aml.batchDelete(export_landTrendr_collection, type = 'imageCollection')

print('Hecho')

done


> Nota: La biblioteca geeViz también proporciona funciones contenedoras para el procesamiento de matrices y la ejecución de LandTrendr. Puedes usar el [`changeDetectionLib.convertToLossGain()`](https://github.com/gee-community/geeViz/blob/fdd8f0080301f8d915214b6e2d50af03a0915777/changeDetectionLib.py#L778C5-L778C22) Función en la biblioteca geeViz para realizar el procesamiento de matrices. También puedes utilizar el [`changeDetectionLib.simpleLANDTRENDR`](https://github.com/gee-community/geeViz/blob/27a0c5d8a0a9c9623e67599bf06448d64b481c56/changeDetectionLib.py#L344) para ejecutar LandTrendr. Consulte ejemplos y documentación en el [geeViz/examples](https://github.com/gee-community/geeViz/blob/master/examples/LANDTRENDRViz.py) repositorio.





#### Convierta la matriz LandTrendr en series de tiempo para ingresarlas en LCMS

Si bien podemos usar la salida de LandTrendr para la detección de cambios basada en umbrales, LCMS la usa como entrada para los modelos de clasificación de cambios supervisados, cobertura del suelo y uso de la tierra. A continuación, convertirá el recurso de imagen de matriz LandTrendr sin procesar en una serie temporal de valores anuales ajustados, de duración del segmento, de magnitud del cambio del segmento y de pendiente.

Este procesamiento se basa en la [`changeDetectionLib.batchSimpleLTFit`](https://github.com/gee-community/geeViz/blob/fdd8f0080301f8d915214b6e2d50af03a0915777/changeDetectionLib.py#L565) función. Esta función convierte resultados de matriz (el formato que estamos usando) o apilados (un formato más antiguo necesario antes de poder exportar matrices de imágenes EE) en una colección de resultados anuales ajustados: por ejemplo, magnitud del cambio, pendiente del cambio, duración de cambio para cada año.

Si bien el valor ajustado de LandTrendr es generalmente de mayor importancia para nuestros modelos, la duración, la pendiente y la magnitud del cambio del segmento de LandTrendr también pueden ayudar a nuestros modelos.

Ejecute el bloque de código a continuación para extraer las producciones anuales ajustadas y verlas en el mapa.

In [ ]:
# Cargar salidas sin procesar de LandTrendr
lt_asset = ee.ImageCollection(f'{pre_baked_path_root}/lcms-training_module-3_landTrendr')

# Convertir en productos anuales ajustados: por ejemplo, magnitud del cambio, pendiente del cambio, duración del cambio para cada año
lt_fit = changeDetectionLib.batchSimpleLTFit(lt_asset,startYear,endYear,None,bandPropertyName='band',arrayMode=True)

# Añadir al mapa
Map.clearMap()
Map.addLayer(lt_fit,{'bands':'swir2_LT_fitted,nir_LT_fitted,red_LT_fitted','min':0.15,'max':0.6},'LandTrendr All Predictors Time Series')


Map.turnOnInspector()
Map.addLayer(studyArea, {'strokeColor': '0000FF'}, "Study Area", False)
Map.view()

Adding layer: LandTrendr All Predictors Time Series
Adding layer: Study Area
Starting webmap
Using default refresh token for geeView: C:\Users\ianho/.config/earthengine/credentials
Local web server at: http://localhost:1233/geeView/ already serving.
cwd c:\RCR\quickLabsTrainingMaterials\lcms-training


#### Inspeccionar
Haga doble clic en el mapa para consultar valores y ver los valores reales y ajustados de los índices a lo largo del tiempo. Visualice la colección de imágenes que se utilizan como predictores en los modelos LCMS. Cuando hace doble clic en la salida en el mapa, observe los diferentes valores disponibles para los modelos.

#### Ejemplo: LandTrendr reduce el ruido en series temporales compuestas originales

La mejor manera de comprender cómo LandTrendr contribuye a reducir el ruido en la serie temporal compuesta original es visualizar las salidas de LandTrendr y la serie compuesta una al lado de la otra.

El siguiente ejemplo toma los valores ajustados de LandTrendr y los muestra junto con los compuestos originales. Observe que LandTrendr ahora llena muchos vacíos. En general, LandTrendr reduce la cantidad de ruido en la serie temporal. Sin embargo, existe el riesgo de ajustar demasiado y omitir cambios de paisaje.

Ejecute el bloque de código siguiente para comparar las series temporales ajustadas con los compuestos de entrada.

Haga clic en el mapa, inspeccione la serie temporal ajustada y observe el timelapse compuesto y el timelapse de Landtrendr.

In [ ]:
# Visualice compuestos landTrendr ajustados
fitted_bns = lt_fit.select(['.*_fitted']).first().bandNames()
out_bns = fitted_bns.map(lambda bn: ee.String(bn).split('_').get(0))

# Dar los mismos nombres que los compuestos.
lt_synth = lt_fit.select(fitted_bns,out_bns)

# Borre la mapa
Map.clearMap()

# Visualice compuestos sin procesar y ajustados con LandTrendr
Map.addTimeLapse(composites,getImagesLib.vizParamsFalse,'Raw Composite Timelapse')
Map.addTimeLapse(lt_synth,getImagesLib.vizParamsFalse,'Fitted LandTrendr Composite Timelapse')

# Unir los valores brutos y ajustados
ltJoined = getImagesLib.joinCollections(composites.select(bandNames),lt_fit.select(['.*_fitted']))

# Añadir al mapa
Map.addLayer(ltJoined,{'min':0.2,'max':1},'Raw and LT Fitted',True)

Map.turnOnInspector()
Map.addLayer(studyArea, {'strokeColor': '0000FF'}, "Study Area", False)
Map.view()

Adding layer: Raw Composite Timelapse
Adding layer: Fitted LandTrendr Composite Timelapse
Adding layer: Raw and LT Fitted
Adding layer: Study Area
Starting webmap
Using default refresh token for geeView: C:\Users\ianho/.config/earthengine/credentials
Local web server at: http://localhost:1233/geeView/ already serving.
cwd c:\RCR\quickLabsTrainingMaterials\lcms-training


## Laboratorio 3 Desafío 1:

Calcule las estadísticas de LandTrendr para NDVI y procese los arreglos hasta el año de cambio más reciente. Agregue la capa a un mapa y visualícela. Titula tu capa "Año de cambio más reciente de LandTrendr NDVI".

**Para los usuarios de Qwiklabs**, se evaluará su finalización en la parte de Seguimiento de actividad del Laboratorio 3.


1. Calcule la salida bruta de LandTrendr para NDVI a partir de los compuestos


2.  Convertir la salida de la matriz al año de cambio más reciente
    * Utilice la siguiente función: `changeDetectionLib.convertToLossGain`
        <br>
        
        Ejemplo:
        ```python
                loss_stack = changeDetectionLib.convertToLossGain(
                lt_array,
                format = 'rawLandTrendr',
                lossMagThresh = -0.15,
                lossSlopeThresh = -0.1,
                gainMagThresh = 0.1,
                gainSlopeThresh = 0.1,
                slowLossDurationThresh = 3,
                chooseWhichLoss = 'newest',
                chooseWhichGain = 'newest',
                howManyToPull = 2)['lossStack'].select(['loss_yr_1'])
                
        ```
        <br>


3.  Extraiga el año de cambio más reciente para la siguiente ubicación.

    * Utilice un punto con estas coordenadas.: `([-65.658,18.294])`

    * Utilice la siguiente función: `g2p.extractPointValuesToDataFrame`
        <br>
        
        Ejemplo:
        ```python
                extracted_values = g2p.extractPointValuesToDataFrame(
                loss_stack,
                ee.Geometry.Point([-65.658,18.294]),
                scale=30,
                crs = "EPSG:5070",
                transform = None,
                reducer = ee.Reducer.first(),
                includeNonSystemProperties = False,
                includeSystemProperties=True
                )
        ```
        <br>
4. Guarde los valores extraídos en un archivo csv.

   * Guarde csv en esta ruta: `"/tmp/challenge/module_3_challenge1_answer.csv"`
     * **Note: The path to the csv must exactly match the path above.**
    <br>
    
    * Creer la `"/tmp/challenge"` carpeta si aún no existe.
      
        Ejemplo:
    ```python
        out_csv = "/tmp/challenge/module_3_challenge1_answer.csv"
        if not os.path.exists(os.path.dirname(out_csv)):os.makedirs(os.path.dirname(out_csv))
    ```
<br>

5.  Checar que el csv de salida existe.
    
    * Ejemplo:
    ```python
        print(os.path.exists(out_csv))
    ```
<br>

In [ ]:
# Inserte el código de desafío aquí

Map.clearMap()

# configurar LandTrendr
test_band = 'NDVI'
run_params['timeSeries'] = composites.select([test_band])
raw_LT = ee.Algorithms.TemporalSegmentation.LandTrendr(**run_params)

# Añadir al mapa
Map.addLayer(raw_LT,{'opacity':0},'LT Raw {}'.format(test_band),True)


# Proporcione una ubicación de ejemplo
pt = ee.Geometry.Point([-65.658,18.294])

# Seleccione la banda de salida de la matriz de imágenes LandTrendr
lt_array = raw_LT.select(['LandTrendr'])


# Convierta para cambiar la pila y seleccione el primer año de pérdida más reciente (el más reciente)
change_stack = changeDetectionLib.convertToLossGain(lt_array,
                                                    format = 'rawLandTrendr',
                                                    lossMagThresh = -0.15,
                                                    lossSlopeThresh = -0.1,
                                                    gainMagThresh = 0.1,
                                                    gainSlopeThresh = 0.1,
                                                    slowLossDurationThresh = 3,
                                                    chooseWhichLoss = 'newest',
                                                    chooseWhichGain = 'newest',
                                                    howManyToPull = 2)['lossStack']

Map.addLayer(change_stack.select(['loss_mag_1']),{'min':-0.8,'max':-0.15,'palette':lossMagPalette},'Loss Magnitude')
Map.addLayer(change_stack.select(['loss_yr_1']),{'min':startYear,'max':endYear,'palette':changeDetectionLib.lossYearPalette},'Loss Year')

Map.turnOnInspector()
Map.view()


# Extraer valores
values = g2p.extractPointValuesToDataFrame(
          change_stack.select(['loss_yr_1']),
          ee.Geometry.Point([-65.658,18.294]),
          scale=30,
          crs = "EPSG:5070",
          transform = None,
          reducer = ee.Reducer.first(),
          includeNonSystemProperties = False,
          includeSystemProperties=True
          )
# Fuardar csv
out_csv = "/tmp/challenge/module_3_challenge1_answer.csv"
if not os.path.exists(os.path.dirname(out_csv)):os.makedirs(os.path.dirname(out_csv))
values.to_csv(out_csv)
display(values)

# Checar que exista csv
print(os.path.exists(out_csv))
print(change_stack.bandNames().getInfo())
print('Hecho')

NameError: name 'run_params' is not defined

### ¡Felicidades! Ya terminó con la parte LandTrendr del Laboratorio 3.

Otros ejemplos de GeeViz LandTrendr:
- https://github.com/gee-community/geeViz/blob/master/examples/LANDTRENDRViz.py
- https://github.com/gee-community/geeViz/blob/master/examples/LANDTRENDRWrapper.py
- https://github.com/gee-community/geeViz/blob/master/examples/LANDTRENDRWrapperNotebook.ipynb


Los datos ajustados de LandTrendr se utilizarán como entradas para LCMS en módulos posteriores.

## 3.4: Escala sobre áreas grandes usando mosaicos

CCDC es el algoritmo con mayor uso de memoria utilizado en LCMS. Como resultado, es más probable que necesite dividir su área de estudio en áreas más pequeñas (mosaicos) u otro enfoque de administración de memoria cuando ejecute CCDC, particularmente en áreas de estudio grandes.

Por lo general, solo utilizará mosaicos cuando un proceso falle debido a errores internos o de memoria. Luego, dividirás el área de estudio en mosaicos. Debe elegir aproximadamente el tamaño máximo de mosaico que permita que su proceso se complete sin errores.

### 3.4.1: Ver mosaicos ejemplo de México

Actualmente, ejecutamos LCMS para los EE. UU. continentales (CONUS), la costa de Alaska, Hawaii y Puerto Rico/Islas Vírgenes de los EE. UU. Para CONUS, tenemos que dividir todo el procesamiento para evitar quedarnos sin memoria.

Aquí te brindamos este ejemplo de escalamiento para que puedas aplicarlo a otras áreas de estudio como México.

El siguiente bloque de código mostrará los mosaicos que los compuestos LCMS utilizan para exportar al activo.

In [ ]:
# Primero, vea los mosaicos utilizados en el flujo de trabajo actual de CONUS LCMS
mexico_composites = ee.ImageCollection('projects/ee-jheyer2325/assets/lcms-training_module-2_composites')\
                                                .filter(ee.Filter.calendarRange(2022,2022,'year'))

# Extrae la geometría de cada mosaico en los compuestos
mexico_composites_tile_geo = mexico_composites.map(lambda f:ee.Feature(f.geometry()).copyProperties(f,['studyAreaName']))

# Agregue los mosaicos y un compuesto como referencia
Map.clearMap()
Map.addLayer(mexico_composites.mosaic(),getImagesLib.vizParamsTrue10k,'Example Mexico 2022 LCMS Composite')
Map.addLayer(mexico_composites_tile_geo,{},'LCMS Mexico Tabasco Tile Geometry')

Map.centerObject(mexico_composites_tile_geo)
Map.turnOnInspector()
Map.view()

Adding layer: Example CONUS 2022 LCMS Composite
Adding layer: LCMS Composite Tile Geometry


Starting webmap
Using default refresh token for geeView: C:\Users\ianho/.config/earthengine/credentials
Starting local web server at: http://localhost:1233/geeView/
HTTP server command: "c:\Python311\python.exe" -m http.server  1233
Done
cwd c:\RCR\quickLabsTrainingMaterials\lcms-training


### 3.4.2: Crea mosaicos de varios tamaños.

**Para determinar qué tamaño de mosaico usaría:** Generalmente, comenzaría con el mosaico más grande posible y avanzaría hacia abajo hasta que deje de tener problemas de memoria. Actualmente, en CONUS LCMS utiliza mosaicos de 480 km (con un búfer de 900 m) para la mayor parte del procesamiento (todo menos CCDC)

Para determinar qué tamaño de mosaico utilizar para su proyecto, puede crear e inspeccionar mosaicos de varios tamaños. A continuación se muestra un ejemplo de cómo crear una pirámide de mosaicos en varias escalas.

Ejecute el bloque de código siguiente para generar un conjunto de cuadrículas de mosaicos de varios tamaños en los Estados Unidos continentales. Active y desactive las capas para comparar el tamaño de las cuadrículas.

In [ ]:
Map.clearMap()

# Establecer área de estudio y proyección.
paises = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1")
mexico_studyArea = paises.filter(ee.Filter.stringContains('ADM0_NAME','Mexico'))
mexico_projection = mexico_composites.first().projection()

# Obtener cuadrícula y agregar al mapa
def getGrid(studyArea,projection,size):
  grid = studyArea.geometry().coveringGrid(projection.atScale(size))
  Map.addLayer(grid,{},f'Tile Grid {size}m')
  return grid

# Obtener rejillas
grid480= getGrid(mexico_studyArea,mexico_projection,480000)
getGrid(mexico_studyArea,mexico_projection,240000)
getGrid(mexico_studyArea,mexico_projection,120000)
getGrid(mexico_studyArea,mexico_projection,60000)

# Añadir al mapa
Map.addLayer(mexico_studyArea,{},'Mexico Rejillas')

Map.turnOnInspector()
Map.view()


Adding layer: Tile Grid 480000m
Adding layer: Tile Grid 240000m
Adding layer: Tile Grid 120000m
Adding layer: Tile Grid 60000m
Adding layer: LCMS CONUS Study Area
Starting webmap
Using default refresh token for geeView: C:\Users\ianho/.config/earthengine/credentials
Local web server at: http://localhost:1233/geeView/ already serving.
cwd c:\RCR\quickLabsTrainingMaterials\lcms-training


#### Conclusiones de este mapa
Puede escalar en todos los tamaños diferentes, pero la mejor práctica para explorar diferentes tamaños de mosaicos es piramidar los mosaicos dividiendo el tamaño entre 2 para cada tamaño más pequeño. La escala de mosaico que elijas para tu operación dependerá de tu área de estudio y del proceso que desees realizar. Los mosaicos pueden ser demasiado grandes o demasiado pequeños para su área de interés según el tamaño del área y la complejidad del proceso que se realiza.

### Cómo usar mosaicos para escalar áreas grandes
Para utilizar mosaicos en la práctica, primero debe crear una lista de cada mosaico disponible. Luego, repetirá la función de interés en cada mosaico, la recortará en el área de estudio, la almacenará en buffer, obtendrá los datos y los exportará.

Ejecute el siguiente código para examinar los dos primeros mosaicos utilizados para México. El script CCDC se ejecutaría sobre cada uno de estos mosaicos uno a la vez y luego combinaría los resultados para crear el resultado en la gran área geográfica de México.

In [ ]:
Map.clearMap()

ids = grid480.limit(2).aggregate_histogram('system:index').keys().getInfo()
for id in ids:
  # Obtenga el mosaico y recórtelo en el área de estudio y luego búfer
  tile = grid480.filter(ee.Filter.eq('system:index',id)).geometry().intersection(mexico_studyArea,240,mexico_projection).buffer(900)
  Map.addLayer(tile,{},'Tile {}'.format(id))

Map.centerObject(tile)
Map.view()

Adding layer: Tile -3,-6
Adding layer: Tile -4,-6
Starting webmap
Using default refresh token for geeView: C:\Users\ianho/.config/earthengine/credentials
Local web server at: http://localhost:1233/geeView/ already serving.
cwd c:\RCR\quickLabsTrainingMaterials\lcms-training


## 3.5: Detección de cambios con CCDC

CCDC tiene una definición fundamentalmente diferente de lo que es "cambio" de LandTrendr. LandTrendr define el cambio como un cambio en la dirección lineal de la serie temporal como se representa con un modelo de regresión lineal, mientras que CCDC define el cambio como un cambio en la estacionalidad (fenología) como se representa mediante la regresión armónica (regresión lineal sobre muchas formas de onda diferentes).

Como resultado, en general, la descripción del cambio que hace LandTrendr se alinea con muchos tipos de cambios relacionados con los bosques, como incendios, insectos y enfermedades, etc. Si bien estos tipos de cambios a menudo cambian la dirección de la trayectoria abruptamente, no siempre cambian la estacionalidad. patrones de manera abrupta.

CCDC puede ser mejor en la detección de cambios que impactan la fenología que LandTrendr puede pasar por alto. Esto puede resultar útil en aplicaciones urbanas, agrícolas y de pastizales.


### 3.5.1: Ejecutando CCDC: un mosaico

#### Configurar mosaicos

Usaremos Tabasco México como área de estudio y ejecutaremos nuestro análisis desde 1984-2023.

Ejecute el bloque de código siguiente para configurar la escala de los mosaicos, el área de estudio, la proyección y la cuadrícula de mosaicos. Agregarás la cuadrícula de mosaicos y el área de estudio al mapa.

In [ ]:
# Establecer el tamaño (en metros) de los mosaicos.
tileSize = 60000

# Especificar área de estudio
paises = ee.FeatureCollection("FAO/GAUL_SIMPLIFIED_500m/2015/level1")
mexico = paises.filter(ee.Filter.stringContains('ADM0_NAME','Mexico'))
studyArea = mexico.filter(ee.Filter.stringContains('ADM1_NAME','Tabasco'))

# Establecer la proyección
projection = mexico_composites.first().projection()

# Obtener la grilla
grid = studyArea.geometry().coveringGrid(projection.atScale(tileSize))

# Borre la mapa
Map.clearMap()

# Anadir al mapa
Map.addLayer(grid,{},'Tile Grid {}m'.format(tileSize))
Map.addLayer(studyArea,{},'Study Area')

Map.turnOnInspector()
Map.centerObject(studyArea)
Map.view()

Adding layer: Tile Grid 60000m
Adding layer: Study Area


Starting webmap
Using default refresh token for geeView: C:\Users\ianho/.config/earthengine/credentials
Local web server at: http://localhost:1233/geeView/ already serving.
cwd c:\RCR\quickLabsTrainingMaterials\lcms-training


### 3.5.2: Obtener imágenes Landsat

A continuación, establezca algunos parámetros preliminares que describan las imágenes Landsat que incorporará al análisis CCDC.

Los parámetros siguientes le resultarán familiares desde el módulo 2. Sin embargo, para este módulo, extraeremos una serie continua de imágenes a las que aplicará una máscara de nube, en lugar de crear composiciones anuales. Esto se debe a que CCDC necesita todas las observaciones claras disponibles, a diferencia de LandTrendr, que necesita compuestos anuales.

Consulte la documentación para [`getImagesLib.getProcessedLandsatScenes`](https://github.com/gee-community/geeViz/blob/27a0c5d8a0a9c9623e67599bf06448d64b481c56/getImagesLib.py#L2563) para más detalles.

#### Seleccionar área de estudio

Comenzaremos con un mosaico en el área de estudio de Tabasco México.

#### Seleccionar rango de fechas

Se debe proporcionar un período de más de 3 años para que los métodos de series temporales funcionen bien. Si proporciona estadísticas precalculadas para cloudScore y TDOM, esto no importa. Aquí ejecutará su análisis desde 1984 hasta 2023.

Actualizará las variables startJulian y endJulian para indicar sus restricciones estacionales. Esto apoya la envoltura para los trópicos y el hemisferio sur. Si usa envoltura y la mayoría de los días ocurren en el segundo año, system:time_start será el 1 de junio del segundo año de manera predeterminada. De lo contrario, todos los system:time_starts se establecerán de forma predeterminada en el 1 de junio del primer año (por ejemplo, si el período de composición es del 1 de diciembre de 2020 al 28 de febrero de 2021, system:time_start se establecerá en el 1 de junio de 2021).

#### Seleccione bandas e índices para obtener

También determinará qué bandas/índices obtener para ejecutar el análisis CCDC. Estos no siempre se utilizarán para buscar interrupciones, como se especifica a continuación en el parámetro `breakpointBands` para CCDC.

Asegúrese de que todas las bandas en el parámetro `ccdcParams.breakpointBands`, que configuraremos a continuación, estén en esta lista.

Las opciones para bandas son: "blue","green","red","nir","swir1","swir2","NDVI","NBR","NDMI","NDSI","brightness","greenness","wetness","fourth","fifth","sixth","tcAngleBG"

#### Eliminar valores altos para bandas e índices.
También escribirá una función para eliminar cualquier valor alto de bandas o índices. Estos valores altos pueden ser artefactos y dar lugar a errores, por lo que los eliminaremos de la serie temporal. Aplicará la función a la imagen.

* Ejecute el bloque de código siguiente para obtener y procesar imágenes Landsat.


In [ ]:
# Enumerar identificadores de mosaicos
ids = grid.aggregate_histogram('system:index').keys().getInfo()

# Obtenga el mosaico y guárdelo para que no falten píxeles en los bordes del mosaico.
tile = grid.filter(ee.Filter.eq('system:index',ids[6]))

# Especifique los años de inicio y finalización de todos los análisis.
startYear = 1984
endYear = 2023

# startJulian: Fecha de inicio juliana
# endJulian: Fecha final juliana
startJulian = 1
endJulian = 365

# Elija si desea incluir Landat 7
# Generalmente solo se incluye cuando los datos son limitados
includeSLCOffL7 = True

# Establecer bandas de exportación
exportBands = ["blue","green","red","nir","swir1","swir2","NDVI"]

# Función de escritura para eliminar cualquier valor de banda/índice extremadamente alto
def removeGT1(img):
  lte1 = img.select(['blue','green','nir','swir1','swir2']).lte(1).reduce(ee.Reducer.min());
  return img.updateMask(lte1);

# Establecer parámetros de visualización
getImagesLib.vizParamsFalse['min']=0.15
getImagesLib.vizParamsFalse['max']=0.8

# Obtener escenas procesadas
processedScenes = getImagesLib.getProcessedLandsatScenes(studyArea = tile, startYear = startYear, endYear = endYear,
                                                    startJulian = startJulian,endJulian = endJulian,
                                                   includeSLCOffL7 = includeSLCOffL7).select(exportBands)

# Aplicar función para eliminar valores de índice/banda alta
processedScenes = processedScenes.map(removeGT1)
# print(processedScenes.size().getInfo())

print('Hecho')

# Añadir al mapa
Map.clearMap()
Map.addLayer(tile,{},'Tile {}'.format(id))
Map.centerObject(tile)

#Map.addLayer(processedScenes,getImagesLib.vizParamsFalse,'Raw Processed Landsat Input')
Map.addLayer(processedScenes,{}, 'Processed Landsat Input')


Map.turnOnInspector()
Map.view()

Get Processed Landsat: 
Start date: Jan 01 1984 , End date: Dec 31 2023
Applying scale factors for C2 L4 data
Applying scale factors for C2 L5 data
Applying scale factors for C2 L8 data
Including All Landsat 7
Applying scale factors for C2 L7 data
Applying scale factors for C2 L9 data
Applying Fmask Cloud Mask
Applying Fmask Shadow Mask
Done
Adding layer: Tile -4,-6
Adding layer: Processed Landsat Input
Starting webmap
Using default refresh token for geeView: C:\Users\ianho/.config/earthengine/credentials
Local web server at: http://localhost:1233/geeView/ already serving.
cwd c:\RCR\quickLabsTrainingMaterials\lcms-training


### 3.5.3: Establecer parámetros CCDC

A continuación, configurará los parámetros que se utilizarán en el algoritmo CCDC. Los parámetros se describen a continuación. Para obtener más información, consulte la [GEE CCDC Documentation](https://developers.google.com/earth-engine/apidocs/ee-algorithms-temporalsegmentation-ccdc).

#### Parámetros CCDC

**Los parámetros CCDC incluyen:**

| Argumento             | Tipo                    | Detalles                                                                                                                                                        |
|----------------------|-------------------------|------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| collection           | ImageCollection         | Colección de imágenes sobre las que ejecutar CCDC.                                                                                           |
| breakpointBands      | List, default: None     | El nombre o índice de las bandas que se utilizarán para la detección de cambios. Si no se especifica, se utilizan todas las bandas.                                     |
| tmaskBands           | List, default: None     | El nombre o índice de las bandas que se utilizarán para la detección iterativa de nubes TMask. Suelen ser la banda verde y la banda SWIR1. Si no se especifica, no se utiliza TMask. Si se especifica, 'tmaskBands' debe incluirse en 'breakpointBands'.                                   |
| minObservations      | Integer, default: 6     | El número de observaciones necesarias para marcar un cambio.                                                                                         |
| chiSquareProbability | Float, default: 0.99    | El umbral de probabilidad de chi-cuadrado para la detección de cambios en el rango de [0, 1]                                                                           |
| minNumOfYearsScaler  | Float, default: 1.33    | Factores de número mínimo de años para aplicar un nuevo accesorio.                                                                                      |
| dateFormat           | Integer, default: 0     | La representación de tiempo que se utilizará durante el ajuste: 0 = jDays, 1 = fracciones de años, 2 = tiempo Unix en milisegundos. De esta forma se codificarán los tiempos de inicio, fin y descanso de cada segmento temporal.                                                             |
| lambda               | Float, default: 20      | Lambda para ajuste de regresión LASSO. Si se establece en 0, se utiliza OLS normal en lugar de LASSO. 20 sería si los datos de entrada estuvieran escalados de 0 a 10000. Si la reflectancia es 0-1, 20 se convertiría en 0.002                                                                 |
| maxIterations        | Entero, predeterminado: 25000 | Número máximo de ejecuciones para la convergencia de regresión LASSO. Si se establece en 0, se utiliza OLS normal en lugar de LASSO.                           |

* Ejecute el siguiente fragmento de código para configurar los parámetros que utilizará en el modelo CCDC.


In [ ]:
# Establecer parámetros CCDC
ccdcParams ={
  'breakpointBands':['green','red','nir','swir1','swir2','NDVI'],
  'tmaskBands' : None,
  'minObservations': 6,
  'chiSquareProbability': 0.99,
  'minNumOfYearsScaler': 1.33,
  'lambda': 0.002, # Puesto que nuestros datos de reflectancia son 0-1 y no 0-10000, dividimos 20 entre 10000
  'maxIterations' : 25000,
  'dateFormat' : 1
}

print('Hecho')

Done


### 3.5.4 - Ejecute el algoritmo CCDC

* Ahora, recorreremos un mosaico y ejecutaremos CCDC.
* Agregará la salida CCDC al mapa, con los datos de Landsat.
* Haga doble clic en las salidas para ver los valores de los datos Landsat y cómo se relacionan con la salida sin procesar del CCDC.
* Notará que la salida CCDC sin procesar es incluso más compleja que la salida LandTrendr.

In [ ]:
#Establecer la colección de escenas en ccdcParams
ccdcParams['collection'] = processedScenes

#Ejecutar CCDC
ccdc = ee.Image(ee.Algorithms.TemporalSegmentation.Ccdc(**ccdcParams))

# Añadir al mapa
Map.addLayer(ccdc,{},'CCDC Output')

Map.turnOnInspector()
Map.view()

Adding layer: CCDC Output
Starting webmap
Using default refresh token for geeView: C:\Users\ianho/.config/earthengine/credentials
Local web server at: http://localhost:1233/geeView/ already serving.
cwd c:\RCR\quickLabsTrainingMaterials\lcms-training


#### Interpretar las salidas CCDC

Para comprender cómo se relacionan las salidas de CCDC con los datos de entrada originales, uniremos los valores de NDVI de entrada sin procesar a los valores de NDVI pronosticados de CCDC.

**Extraer los datos puede llevar algún tiempo y consultar este mapa a menudo generará errores, ya que este proceso requiere bastante procesamiento computacional. Por favor sea paciente.**

In [ ]:
# Especifique qué bandas mostrar en el ejemplo.
exampleBandNames = ['NDVI']

# Ahora unámonos al CCDC sin procesar y previsto por un subconjunto de tiempo.
processedScenes = processedScenes\
                    .filter(ee.Filter.calendarRange(2010,2023,'year'))\
                    .map(getImagesLib.addYearYearFractionBand)

# Si se deben llenar los espacios entre el año final de los segmentos y el año de inicio posterior a la fecha de ruptura
fillGaps = False

fitted = changeDetectionLib.predictCCDC(ccdc,processedScenes.select(['year']),fillGaps=fillGaps,whichHarmonics=[1,2,3])

exampleFittedBandNames = [f'{bn}_CCDC_fitted' for bn in exampleBandNames]

ccdcJoined = getImagesLib.joinCollections(processedScenes.select(exampleBandNames),fitted.select(exampleFittedBandNames))
ccdcJoinedBns = ccdcJoined.first().bandNames().getInfo()

# View the map
Map.clearMap()
Map.addLayer(ccdcJoined,{'min':0.2,'max':0.8},'Raw Landsat and CCDC Fitted')
Map.turnOnInspector()
Map.view()



Adding layer: Raw Landsat and CCDC Fitted
Starting webmap
Using default refresh token for geeView: C:\Users\ianho/.config/earthengine/credentials
Local web server at: http://localhost:1233/geeView/ already serving.
cwd c:\RCR\quickLabsTrainingMaterials\lcms-training


#### Ver salidas CCDC en un solo punto

Extraeremos un solo píxel de la salida para ilustrar cómo se relacionan las entradas sin procesar con la salida CCDC instalada. Esto puede llevar algún tiempo.

In [ ]:
# # Proporcione una ubicación de ejemplo
pt = ee.Geometry.Point([-93.63546,17.78971])

# Extraer los valores y trazarlos.
print('Extracting raw Landsat and fitted CCDC values')
crs = mexico_composites.first().projection()
timSeries = g2p.extractPointValuesToDataFrame(ccdcJoined,pt,scale=30,crs = crs, transform = None)
timSeries['system:time_start']= g2p.pandas.to_datetime(timSeries['system:time_start'], unit='ms')

timSeriesT = timSeries[ccdcJoinedBns]
timSeriesT.index = timSeries['system:time_start']
timSeriesT.plot.line(title='Raw Landsat and CCDC Fitted',xlabel='Date',ylabel='Value')

print('Hecho')

NameError: name 'ee' is not defined

#### Inspeccionar

¿Qué observa sobre los valores brutos de NDVI y los valores de CCDC? ¿Cuántos segmentos se colocan? Recuerde que se ajusta una salida con formato similar para cada píxel de la imagen, para cada una de las bandas que seleccionó para incluir en el algoritmo.

### 3.5.5 - Ejecute CCDC en todos los mosaicos
**Iterar en todos los mosaicos y exportar salidas CCDC**

Ahora que comprende cómo ejecutar CCDC sobre datos de Landsat, ejecutará el mismo proceso en todos los mosaicos y exportará los datos como un activo.

Ejecute el bloque de código siguiente para iterar sobre los identificadores de mosaico para obtener imágenes, ejecutar el algoritmo CCDC y exportar el resultado.


In [ ]:
# Iterar sobre identificadores
for id in ids:
    print(id)
    # Obtenga el mosaico y guárdelo para que no falten píxeles en los bordes del mosaico.
    tile = grid.filter(ee.Filter.eq('system:index',id)).geometry().intersection(studyArea,240,projection).buffer(900)

    # Map.addLayer(tile,{},'Tile {}'.format(id))

    processedScenes = getImagesLib.getProcessedLandsatScenes(studyArea = tile,startYear = startYear, endYear = endYear,
                                                        startJulian = startJulian,endJulian = endJulian,
                                                        includeSLCOffL7 = includeSLCOffL7).select(exportBands)
    processedScenes = processedScenes.map(removeGT1)
    # print(processedScenes.size().getInfo())

    # Establecer la colección de escenas en ccdcParams
    ccdcParams['collection'] = processedScenes

    # Ejecutar CCDC
    ccdc = ee.Image(ee.Algorithms.TemporalSegmentation.Ccdc(**ccdcParams))
    ccdc = ccdc.set({'startYear':startYear,
                     'endYear':endYear,
                     'startJulian':startJulian,
                     'endJulian':endJulian,
                     'TileSize':tileSize,
                     'TileID':id})

    # Exportar la salida
    exportName = 'CCDC_Tile-{}m_ID{}_yrs{}-{}_jds{}-{}'.format(tileSize,id.replace(',','-'),startYear,endYear,startJulian,endJulian)
    exportPath = f'{export_ccdc_collection}/{exportName}'
    print(exportPath)

    getImagesLib.exportToAssetWrapper(ccdc,exportName,exportPath,{'.default':'sample'},tile,30,crs,None,overwrite=False)

print('Hecho')

NameError: name 'ids' is not defined

#### Seguimiento de tareas y gestión de archivos.

Puede realizar un seguimiento de las tareas en el bloque de código a continuación o visitando https://code.earthengine.google.com/tasks .

Descomente los siguientes comandos para realizar un seguimiento de las tareas, si lo desea. Esto informará qué tareas están en proceso y su estado de exportación.


In [ ]:
# Puede realizar un seguimiento de las tareas aquí o en https://code.earthengine.google.com/tasks
# Si desea realizar un seguimiento de las tareas, utilice esto:
# tml.trackTasks2()

# Si desea cancelar todas las tareas en ejecución, puede utilizar esta función
# tml.batchCancel()

# Si desea vaciar la colección de todas las imágenes
# aml.batchDelete(exportPathRoot, type = 'imageCollection')

print('Hecho')

done


#### Inspeccionar salidas

Incorpora los resultados y crea un mosaico en una sola imagen. Usaremos esta imagen en módulos posteriores.

In [ ]:
Map.clearMap()

# Incorpora los resultados y crea un mosaico en una sola imagen.
ccdcImg = ee.ImageCollection(f'{pre_baked_path_root}/lcms-training_module-3_CCDC').mosaic()
Map.addLayer(ccdcImg,{'addToLegend':False},'CCDC Raw Image')
Map.centerObject(studyArea,10)
Map.turnOnInspector()
Map.view()

Adding layer: CCDC Raw Image
Starting webmap
Using default refresh token for geeView: C:\Users\ianho/.config/earthengine/credentials
Local web server at: http://localhost:1233/geeView/ already serving.
cwd c:\RCR\quickLabsTrainingMaterials\lcms-training


Haga clic en el mapa para consultar las salidas. Verás que hay salidas multibanda que incluyen coeficientes, magnitud y RMSE para todas las bandas. Al igual que las salidas de LandTrendr, las salidas de la matriz son difíciles de analizar. Visualizaremos los resultados con más detalle un poco más adelante.


### 3.5.6: Uso de CCDC para detectar cambios

Puede utilizar las pausas de CCDC como herramienta para detectar cambios significativos en la estacionalidad (fenología).


In [ ]:
## INSPECCIONAR PROPIEDADES DE CCDCIMG
print(ccdcImg.bandNames().getInfo())

['tStart', 'tEnd', 'tBreak', 'numObs', 'changeProb', 'blue_coefs', 'green_coefs', 'red_coefs', 'nir_coefs', 'swir1_coefs', 'swir2_coefs', 'NDVI_coefs', 'blue_rmse', 'green_rmse', 'red_rmse', 'nir_rmse', 'swir1_rmse', 'swir2_rmse', 'NDVI_rmse', 'blue_magnitude', 'green_magnitude', 'red_magnitude', 'nir_magnitude', 'swir1_magnitude', 'swir2_magnitude', 'NDVI_magnitude']


#### Inspeccionar la función de detección de cambios ccdc

A continuación, inspeccione la función `ccdcChangeDetection` en `changeDetectionLib` para ver qué tipo de parámetros y entradas necesitaremos.

Presta atención a:
- a) los objetos/parámetros que se ingresan a la función y
- b) lo que devuelve la función

In [ ]:
# opción de impresión
print(inspect.getsource(changeDetectionLib.ccdcChangeDetection))

def ccdcChangeDetection(ccdcImg,bandName):
  magKeys = ['.*_magnitude']
  tBreakKeys = ['tBreak']
  changeProbKeys = ['changeProb']
  changeProbThresh = 1

  #Pull out pieces from CCDC output
  magnitudes = ccdcImg.select(magKeys)
  breaks = ccdcImg.select(tBreakKeys)
  
  #Map.addLayer(breaks.arrayLength(0),{'min':1,'max':10});
  changeProbs = ccdcImg.select(changeProbKeys)
  changeMask = changeProbs.gte(changeProbThresh)
  magnitudes = magnitudes.select(bandName + '.*')

  
  #Sort by magnitude and years
  breaksSortedByMag = breaks.arraySort(magnitudes)
  magnitudesSortedByMag = magnitudes.arraySort()
  changeMaskSortedByMag = changeMask.arraySort(magnitudes)
  
  breaksSortedByYear = breaks.arraySort()
  magnitudesSortedByYear = magnitudes.arraySort(breaks)
  changeMaskSortedByYear = changeMask.arraySort(breaks)
  
  #Get the loss and gain years and magnitudes for each sorting method
  highestMagLossYear = breaksSortedByMag.arraySlice(0,0,1).arrayFlatten([['loss_year']])
  highestM

#### Conclusiones de la función ccdcChangeDetection
Tenga en cuenta que la función devuelve un objeto que contiene un diccionario con objetos titulados `mostRecent` y `highestMag`. Estas salidas son métodos diferentes para ordenar la información de cambios de CCDC: por ruptura de CCDC más reciente ("mostRecent") o de mayor magnitud ("highestMag").

### 3.5.7: Establecer parámetros de detección de cambios para el algoritmo CCDC
Esta función nos permite manipular las salidas CCDC en formato de matriz para obtener información significativa, es decir, información que es más directamente útil que una trayectoria espectral modelada.

#### Especificar la banda utilizada para la detección de cambios

Esto es muy importante para la magnitud de las pérdidas y ganancias, ya que el año del cambio será el mismo para todos los años.

`changeDetectionBandName = 'NDVI'`

#### Especificar el método de clasificación para mostrar el cambio

Elige si mostrar el más reciente (`'mostRecent'`) o magnitud más alta (`'highestMag'`) CCDC rompe

In [ ]:
#Especifique qué banda usar para pérdidas y ganancias.
#This is most important for the loss and gain magnitude since the year of change will be the same for all years
changeDetectionBandName = 'NDVI'

# Elija si desea mostrar el CCDC rompe más reciente ('mostRecent') o de mayor magnitud ('highestMag') CCDC rompe
sortingMethod = 'mostRecent'

### 3.5.8: Ejecute la detección de cambios CCDC e inspeccione las salidas

Ahora veremos formas más útiles de visualizar los resultados del CCDC.

Primero, extraeremos los años y la magnitud del cambio, según el método de clasificación que acaba de seleccionar; el valor predeterminado en este cuaderno es el cambio "más reciente".

Esto creará cuatro capas para agregar al mapa:
- Año de pérdida más reciente
- Magnitud de la pérdida más reciente
- Año de ganancia más reciente
- Magnitud de ganancia más reciente

Haga doble clic en el mapa para ver años sin procesar de pérdidas y ganancias. Active también las capas de magnitud para ver la magnitud del cambio que ocurrió en esos años.

Observe que a medida que acerca el zoom, las capas cambian: GEE procesa las salidas a un nivel de zoom establecido y recalcula a medida que acerca o aleja el zoom.

In [ ]:
# Extraer salidas de detección de cambios de las salidas CCDC para la banda seleccionada
changeObj = changeDetectionLib.ccdcChangeDetection(ccdcImg,changeDetectionBandName);

# Borre la mapa
Map.clearMap()

# Agregar nuevas capas al mapa
Map.addLayer(changeObj[sortingMethod]['loss']['year'],{'min':startYear,'max':endYear,'palette':changeDetectionLib.lossYearPalette},sortingMethod + ' Loss Year')
Map.addLayer(changeObj[sortingMethod]['loss']['mag'],{'min':-0.5,'max':-0.1,'palette':changeDetectionLib.lossMagPalette},sortingMethod + ' Loss Magnitude',False);
Map.addLayer(changeObj[sortingMethod]['gain']['year'],{'min':startYear,'max':endYear,'palette':changeDetectionLib.gainYearPalette},sortingMethod + ' Gain Year');
Map.addLayer(changeObj[sortingMethod]['gain']['mag'],{'min':0.05,'max':0.2,'palette':changeDetectionLib.gainMagPalette},sortingMethod + ' Gain Magnitude',False);

Map.centerObject(studyArea,10)
Map.turnOnInspector()
Map.view()


Adding layer: mostRecent Loss Year
Adding layer: mostRecent Loss Magnitude
Adding layer: mostRecent Gain Year
Adding layer: mostRecent Gain Magnitude
Starting webmap
Using default refresh token for geeView: C:\Users\ianho/.config/earthengine/credentials
Local web server at: http://localhost:1233/geeView/ already serving.
cwd c:\RCR\quickLabsTrainingMaterials\lcms-training


### 3.5.9: Obtenga compuestos sintéticos y coeficientes armónicos.

Para utilizar CCDC para la clasificación, será necesario manipular la imagen de la matriz para obtener datos significativos, como compuestos sintéticos y coeficientes armónicos. La función `changeDetectionLib.getCCDCSegCoeffs` realiza esta manipulación de matriz para extraer los compuestos y coeficientes. Inspeccione la función a continuación.

#### Función de inspección de coeficientes

Presta atención a:
- a) los parámetros de entrada a la función
- b) Qué sucede si fillGaps = True vs si fillGaps = False
- c) donde los coeficientes se almacenan en la imagen ccdc de entrada
- d) cómo se procesan los coeficientes desde su formato de entrada en bandas de imágenes únicas

In [ ]:
# inspeccionar la función de coeficientes del segmento
print(inspect.getsource(changeDetectionLib.getCCDCSegCoeffs) )


def getCCDCSegCoeffs(timeImg,ccdcImg,fillGaps):
  coeffKeys = ['.*_coefs']
  tStartKeys = ['tStart']
  tEndKeys = ['tEnd']
  tBreakKeys = ['tBreak']
  
  #Get coeffs and find how many bands have coeffs
  coeffs = ccdcImg.select(coeffKeys)
  bns = coeffs.bandNames()
  nBns = bns.length()
  harmonicTag = ee.List(['INTP','SLP','COS1','SIN1','COS2','SIN2','COS3','SIN3'])

   
  #Get coeffs, start and end times
  coeffs = coeffs.toArray(2)
  tStarts = ccdcImg.select(tStartKeys)
  tEnds = ccdcImg.select(tEndKeys)
  tBreaks = ccdcImg.select(tBreakKeys)
  
  #If filling to the tBreak, use this
  tStarts = ee.Image(ee.Algorithms.If(fillGaps,tStarts.arraySlice(0,0,1).arrayCat(tBreaks.arraySlice(0,0,-1),0),tStarts))
  tEnds = ee.Image(ee.Algorithms.If(fillGaps,tBreaks.arraySlice(0,0,-1).arrayCat(tEnds.arraySlice(0,-1,None),0),tEnds))
  
  
  #Set up a mask for segments that the time band intersects
  tMask = tStarts.lt(timeImg).And(tEnds.gte(timeImg)).arrayRepeat(1,1).arrayRepeat(2,1)
  coeffs = 

#### Función de coeficientes de ejecución

A continuación, configurará los parámetros para la función `changeDetectionLib.getCCDCSegCoeffs` para extraer los coeficientes.

Aquí, simplemente ejecutará el código para extraer los coeficientes una sola vez. La siguiente función `ee.Image()` crea una imagen donde cada píxel tiene el mismo valor: 2015.5. Esta imagen se utiliza como entrada a la función para extraer los coeficientes en esos valores de tiempo particulares.

Ejecute el bloque de código a continuación para obtener los coeficientes y agregarlos al mapa. Puedes hacer clic en el mapa para consultar las salidas. A continuación, inspeccionaremos las salidas en formato de matriz para comprender cómo manipulamos las salidas de la matriz en imágenes que podamos visualizar y usar para la clasificación.

In [ ]:
# establecer parámetros y obtener coeficientes de segmento
fillGaps = False
segCoeffs = changeDetectionLib.getCCDCSegCoeffs(ee.Image(2015.5), ccdcImg, fillGaps)

# inspeccionar nueva salida
print(segCoeffs.bandNames().getInfo())

# Añadir al mapa
Map.clearMap()
Map.addLayer(segCoeffs, {}, 'Seg Coeffs')
Map.addLayer(ccdcImg, {}, 'Raw Img')

Map.turnOnInspector()
Map.view()

['constant', 'blue_coefs_INTP', 'blue_coefs_SLP', 'blue_coefs_COS1', 'blue_coefs_SIN1', 'blue_coefs_COS2', 'blue_coefs_SIN2', 'blue_coefs_COS3', 'blue_coefs_SIN3', 'green_coefs_INTP', 'green_coefs_SLP', 'green_coefs_COS1', 'green_coefs_SIN1', 'green_coefs_COS2', 'green_coefs_SIN2', 'green_coefs_COS3', 'green_coefs_SIN3', 'red_coefs_INTP', 'red_coefs_SLP', 'red_coefs_COS1', 'red_coefs_SIN1', 'red_coefs_COS2', 'red_coefs_SIN2', 'red_coefs_COS3', 'red_coefs_SIN3', 'nir_coefs_INTP', 'nir_coefs_SLP', 'nir_coefs_COS1', 'nir_coefs_SIN1', 'nir_coefs_COS2', 'nir_coefs_SIN2', 'nir_coefs_COS3', 'nir_coefs_SIN3', 'swir1_coefs_INTP', 'swir1_coefs_SLP', 'swir1_coefs_COS1', 'swir1_coefs_SIN1', 'swir1_coefs_COS2', 'swir1_coefs_SIN2', 'swir1_coefs_COS3', 'swir1_coefs_SIN3', 'swir2_coefs_INTP', 'swir2_coefs_SLP', 'swir2_coefs_COS1', 'swir2_coefs_SIN1', 'swir2_coefs_COS2', 'swir2_coefs_SIN2', 'swir2_coefs_COS3', 'swir2_coefs_SIN3', 'NDVI_coefs_INTP', 'NDVI_coefs_SLP', 'NDVI_coefs_COS1', 'NDVI_coefs_SIN1'

### 3.5.10: Manipular la matriz en salidas de imágenes utilizables

A continuación, manipulará las matrices masivas para convertirlas en resultados de imágenes utilizables. Como ejemplo, inspeccionaremos cómo se ve la matriz en un solo punto.

El primer paso es establecer el punto de ejemplo y extraer los coeficientes, nombres de bandas y armónicos de la salida CCDC.

Ejecute el bloque de código siguiente para extraer los datos de la imagen CCDC. Luego, visualizará la salida de un segmento en el punto de entrada.

In [ ]:
# Proporcione una ubicación de ejemplo
pt = ee.Geometry.Point([-93.63546,17.78971])

# nombres de atributos
coeffKeys = ['.*_coefs']
tStartKeys = ['tStart']
tEndKeys = ['tEnd']
tBreakKeys = ['tBreak']

# Obtenga coeficientes y encuentre cuántas bandas tienen coeficientes
coeffs = ccdcImg.select(coeffKeys)
bns = coeffs.bandNames().getInfo()
input_bns = [bn.split('_')[0] for bn in bns]
harmonicTag = ['INTP','SLP','COS1','SIN1','COS2','SIN2','COS3','SIN3']


# Obtener coeficientes, horas de inicio y finalización.
coeffs = coeffs.toArray(2)
tStarts = ccdcImg.select(tStartKeys)
tEnds = ccdcImg.select(tEndKeys)
tBreaks = ccdcImg.select(tBreakKeys)

index = [f'Segment {i}' for i in list(range(1,5))]

print("Hecho")

# Visualizar la salida del ejemplo
display(g2p.imageArrayPixelToDataFrame(coeffs.arraySlice(1,0,1).arraySlice(0,0,1).arrayProject([2,1]), pt,None,crs,transform, 'Intercept',index = ['Segment 1'],columns =input_bns ))

Done


,blue,green,red,nir,swir1,swir2,NDVI
Segment 1,2.883502,3.282841,1.889072,19.140961,5.329586,1.267390,0.760559


#### Inspeccionar las salidas de la matriz

Cada píxel tiene cualquier número de segmentos. Para cada segmento, existe un modelo para cada banda.
Por ejemplo, las intersecciones de la ecuación para cada banda del primer segmento se muestran arriba.

A continuación, ejecute el bloque de código siguiente para observar todos los coeficientes de un píxel determinado para todas las bandas y todos los segmentos.


In [ ]:
# mostrar todos los coeficientes para un solo píxel
display(g2p.imageArrayPixelToDataFrame(coeffs, pt,None,crs,transform, 'Coeffs',columns = harmonicTag,index=index))

,INTP,SLP,COS1,SIN1,COS2,SIN2,COS3,SIN3
Segment 1,"[2.8835019637837616, 3.2828407883810056, 1.8890716245408166, 19.14096117316336, 5.32958581685124, 1.2673900698228309, 0.7605586342114968]","[-0.0014289784894894713, -0.0016160401283955837, -0.000923589550535398, -0.009436630923169133, -0.0025919058730427776, -0.0006002759631661662, 0]","[-0.01186843936476042, -0.02098628819978426, -0.01571539305900891, -0.056630206538701544, -0.020291496499357173, -0.008427349310669975, 0.04611739888770422]","[0, 0, 0, -0.004567996992550899, 0.003230075007047745, 0, 0.009124222149743245]","[0, -0.0016317531211471504, -0.001774852987395223, -0.005093326960209953, -0.02064884629845176, -0.011918990964258388, 0.00213493267705074]","[0, 0, 0, 0, 0, 0, 0.01964705696807787]","[0.002384153070020832, 0.003457602225186136, 0.0025340059036947633, 0, 0, 0.0009508011613858051, -0.025616353056236732]","[0.002910448220751587, 0.005529002494703668, 0.004594814712743522, 0.00042706699447165783, 0.009437693570915878, 0.0030991824168144955, -0.03118448910164181]"
Segment 2,"[-8.178675304391675, -8.567455788409305, -8.585631842168647, -10.887516806370494, 0.42282078696136705, 0.28414772604338856, 0.1243622199456729]","[0.004146312252314901, 0.004394562620758038, 0.004441637236630287, 0.00563385399970798, 0, 0, 0]","[-0.015031742753393139, -0.018733700849538595, -0.015736692525579243, -0.017957310568418212, -0.013183662346069571, -0.014953523384902585, 0]","[0.01045798256411022, 0.018567807032632097, 0.021002172225181297, 0.015381522615022576, 0.04884627085686614, 0.05317092519847767, -0.009940719367876655]","[-9.026335174490329e-05, 0, 0, 0, -0.011864347346887232, -0.008475299950807794, 0]","[0.0001973748731300362, 0.00020375191168091125, 0, 0.007281413879694335, 0.033224318510948876, 0.03621641435912726, 0.005348317025927302]","[-0.010136155719827131, -0.017678294046348877, -0.023494124785338872, -0.010202604645432222, -0.028253406666658296, -0.03274715185488118, 0.020911638035406462]","[0.007635588989686705, 0.008221749753545272, 0.007934732692333683, 0.011885234801459157, 0, 0, 0]"
Segment 3,"[40.921006545882534, 50.895096297031586, 69.48867809045338, 0.8834294022409347, 76.92873468727524, 59.97626481008383, -183.77273951155982]","[-0.020259890824980097, -0.02518305107288108, -0.03440506306050126, -0.00023969860291634586, -0.03800332636106635, -0.02966235885704887, 0.09142888258873613]","[0, 0, -0.0024225947032943487, -0.0016084047433077091, -0.016494180170297905, -0.013406892788657997, 0.02902326656997849]","[0.005854176136300245, 0.00397697856325619, 0.007443825668502859, -0.0076462463157694425, 0.0021441131566921037, 0.002351250810332423, -0.012174527610339603]","[-0.014370010048930525, -0.013268489602268415, -0.01680896003632936, -0.0036527915819201755, -0.019796965920781923, -0.013964558531023071, 0.03579450237639679]","[0.008811111586499342, 0.012618469843064676, 0.01361870695976215, 0.024766213847678697, 0.021772726540925325, 0.014124435464744166, -0.0065829842625007]","[-0.020052238813802078, -0.015932060670294356, -0.01707310212186034, 0, -0.004396921947604966, -0.0042026664281565864, 0.026255827857797617]","[0, 5.383251312164806e-06, 0, 0, 0, 0, -0.005891841365245101]"
Segment 4,"[-81.05418526482875, -118.01267002729952, -158.2880580670942, -30.30693193427896, -196.03929233869582, -162.25359177311236, 315.4606911372531]","[0.040157854624715446, 0.058484343822889764, 0.07842674538386808, 0.015224468791355944, 0.09720045762482044, 0.08041071502669586, -0.15590816344669936]","[0.007472415860416005, 0.014965945315045614, 0.028236782931977088, 0, 0.0456677755800225, 0.04424488962356279, -0.08085708479463748]","[0.0005224995743545358, 0, 0.0019928640089984374, -0.0047594782098874374, 0, 0, -0.005990266176790861]","[-0.007480593138068563, -0.014385875825021191, -0.019532340820599563, -0.02110208150576144, -0.05293998042020171, -0.037081218150552306, 0.00983054564983115]","[-0.0047991985520175195, -0.008273657115843325, -0.013458547981285734, 0.009891817020

#### Cómo utilizar estas salidas

Entonces, ¡hay MUCHOS datos incluso para un solo píxel! Puede ver por qué el algoritmo de modelado CCDC consume tanta memoria. Ahora bien, ¿cómo se pueden hacer útiles estos resultados?

Uno de los usos más comunes de CCDC es elegir una fecha específica y encontrar el modelo con esa fecha. Del modelo para esa fecha, puede obtener los coeficientes y los valores pronosticados del modelo.

#### Inspeccionar la información de la fecha del segmento

Para hacer esto, el primer paso es determinar en qué segmento se encuentra una fecha específica.

Ejecute el bloque de código siguiente para ver la estructura de la información de fecha para cada segmento.

In [ ]:
display(g2p.imageArrayPixelToDataFrame(tStarts.arrayCat(tEnds,1),\
                                       pt,None,crs,transform, 'Segment Start and End Dates',index=index,columns = ['tStart','tEnd']))
print()

,tStart,tEnd
Segment 1,1984.813408,1992.632670
Segment 2,1999.729263,2013.309009
Segment 3,2013.330923,2020.145418
Segment 4,2020.164595,2021.653986
Segment 5,2021.766226,2023.671770


`tStart` y `tEnd` son la fecha de inicio y finalización de cada segmento. Observe que las fechas se muestran en fracciones de año. Observe también que el tEnd del segmento 1 es varios años antes del tStart del segmento 2. Esto se debe a que CCDC necesita varias observaciones para iniciar el siguiente modelo armónico (segmento).

#### Inspeccionar rompes

A continuación, inspeccione los rompes. Las tBreaks son la fecha en que hubo una desviación significativa y el modelo armónico se detuvo.

Ejecute el bloque de código siguiente para ver las rupturas de este píxel en particular.


In [ ]:
# Inspeccionar rompes
display(g2p.imageArrayPixelToDataFrame(tBreaks, pt,None,crs,transform, 'tBreaks',index=index,columns = ['tBreaks']))

,tBreaks
Segment 1,1999.729263
Segment 2,2013.330923
Segment 3,2020.164595
Segment 4,2021.766226
Segment 5,2023.712849


Observe que hay una pausa en el último segmento.
Este no es siempre el caso, sin embargo. A veces hay una pausa hacia el final de la serie temporal y no hay suficientes observaciones para comenzar un nuevo segmento antes del final.

#### Encuentra el segmento que cruza una fecha determinada

Ahora, encontraremos el segmento que cruza una fecha determinada. Elegirá una fecha de interés y devolverá un arry con un 1 para los segmentos que la fecha intersecta y un 0 para los segmentos que la fecha no intersecta.


In [ ]:
# Ahora, encontraremos el segmento que cruza una fecha determinada.
date = 2017.9
timeBandName = 'year'
timeImg = ee.Image(date).rename([timeBandName])

# Encuentra qué segmento se cruza
tMask = tStarts.lt(timeImg).And(tEnds.gte(timeImg))

# mostrar resultados
display(g2p.imageArrayPixelToDataFrame(tMask,\
                                       pt,None,crs,transform, 'Segment date intersects',index = index))

,0
Segment 1,0
Segment 2,0
Segment 3,1
Segment 4,0
Segment 5,0


A continuación, vuelva a formatear la máscara para que pueda aplicarse en modelos de múltiples bandas.

In [ ]:
# Vuelva a formatear la máscara para que pueda aplicarse en varios modelos de banda.
tMask = tMask.arrayRepeat(1,1).arrayRepeat(2,1)

# mostrar
display(g2p.imageArrayPixelToDataFrame(tMask,\
                                     pt,None,crs,transform, 'Segment date intersects reformatted',index=index))

,0
Segment 1,[0]
Segment 2,[0]
Segment 3,[1]
Segment 4,[0]
Segment 5,[0]


Enmascare los coeficientes para el segmento que cruza el tiempo. Finalmente, enmascare para que solo quede el segmento de interés.

In [ ]:
# Enmascare los coeficientes para el segmento que cruza el tiempo
coeffsSeg = coeffs.arrayMask(tMask)

# mostrar
display(g2p.imageArrayPixelToDataFrame(coeffsSeg,\
                                       pt,None,crs,transform, 'Coeffs Masked',index=[f'Segment {date}'],columns = harmonicTag ))


,INTP,SLP,COS1,SIN1,COS2,SIN2,COS3,SIN3
Segment 2017.9,"[40.921006545882534, 50.895096297031586, 69.48867809045338, 0.8834294022409347, 76.92873468727524, 59.97626481008383, -183.77273951155982]","[-0.020259890824980097, -0.02518305107288108, -0.03440506306050126, -0.00023969860291634586, -0.03800332636106635, -0.02966235885704887, 0.09142888258873613]","[0, 0, -0.0024225947032943487, -0.0016084047433077091, -0.016494180170297905, -0.013406892788657997, 0.02902326656997849]","[0.005854176136300245, 0.00397697856325619, 0.007443825668502859, -0.0076462463157694425, 0.0021441131566921037, 0.002351250810332423, -0.012174527610339603]","[-0.014370010048930525, -0.013268489602268415, -0.01680896003632936, -0.0036527915819201755, -0.019796965920781923, -0.013964558531023071, 0.03579450237639679]","[0.008811111586499342, 0.012618469843064676, 0.01361870695976215, 0.024766213847678697, 0.021772726540925325, 0.014124435464744166, -0.0065829842625007]","[-0.020052238813802078, -0.015932060670294356, -0.01707310212186034, 0, -0.004396921947604966, -0.0042026664281565864, 0.026255827857797617]","[0, 5.383251312164806e-06, 0, 0, 0, 0, -0.005891841365245101]"


La matriz anterior enumera todos los coeficientes de todas las bandas. Entonces, nuestro siguiente paso es convertir esta matriz en una imagen multibanda.

#### Convertir a imagen multibanda

Ahora que encontramos qué segmento intersecta la fecha, necesitamos reformatear la salida en una imagen multibanda más fácil de usar.

Primero, proyectaremos los ejes para que solo haya 2 dimensiones para un píxel determinado en lugar de 3. Es decir, estamos eliminando la dimensión del tiempo para que las dimensiones restantes sean: coeficiente y banda.



In [ ]:
coeffsSeg = coeffs.arrayMask(tMask)

# Utilice arrayProject para tomar la tercera dimensión y convertirla en la primera.
coeffsSeg = coeffsSeg.arrayProject([2,1])

# mostrar
display(g2p.imageArrayPixelToDataFrame(coeffsSeg,\
                                       pt,None,crs,transform, 'Coeffs Masked Reformatted',index = harmonicTag ,columns = bns))

,blue_coefs,green_coefs,red_coefs,nir_coefs,swir1_coefs,swir2_coefs,NDVI_coefs
INTP,40.921007,50.895096,69.488678,0.883429,76.928735,59.976265,-183.772740
SLP,-0.020260,-0.025183,-0.034405,-0.000240,-0.038003,-0.029662,0.091429
COS1,0.000000,0.000000,-0.002423,-0.001608,-0.016494,-0.013407,0.029023
SIN1,0.005854,0.003977,0.007444,-0.007646,0.002144,0.002351,-0.012175
COS2,-0.014370,-0.013268,-0.016809,-0.003653,-0.019797,-0.013965,0.035795
SIN2,0.008811,0.012618,0.013619,0.024766,0.021773,0.014124,-0.006583
COS3,-0.020052,-0.015932,-0.017073,0.000000,-0.004397,-0.004203,0.026256
SIN3,0.000000,0.000005,0.000000,0.000000,0.000000,0.000000,-0.005892


#### Transponer la matriz
Esto se hace para que el orden final de las bandas sea agrupado por banda y no por nombre de coeficiente del modelo.

In [ ]:
# transponer la matriz
coeffsSeg = coeffsSeg.arrayTranspose(1,0)

# mostrar
display(g2p.imageArrayPixelToDataFrame(coeffsSeg,\
                                       pt,None,crs,transform, 'Coeffs Masked Reformatted Transposed',index = bns,columns = harmonicTag))

,INTP,SLP,COS1,SIN1,COS2,SIN2,COS3,SIN3
blue_coefs,40.921007,-0.020260,0.000000,0.005854,-0.014370,0.008811,-0.020052,0.000000
green_coefs,50.895096,-0.025183,0.000000,0.003977,-0.013268,0.012618,-0.015932,0.000005
red_coefs,69.488678,-0.034405,-0.002423,0.007444,-0.016809,0.013619,-0.017073,0.000000
nir_coefs,0.883429,-0.000240,-0.001608,-0.007646,-0.003653,0.024766,0.000000,0.000000
swir1_coefs,76.928735,-0.038003,-0.016494,0.002144,-0.019797,0.021773,-0.004397,0.000000
swir2_coefs,59.976265,-0.029662,-0.013407,0.002351,-0.013965,0.014124,-0.004203,0.000000
NDVI_coefs,-183.772740,0.091429,0.029023,-0.012175,0.035795,-0.006583,0.026256,-0.005892


#### Convertir matriz en imagen multibanda

El método arrayFlatten permite convertir una imagen de matriz en una imagen multibanda para facilitar su uso. Condensaremos la matriz bidimensional anterior en una lista unidimensional que se adjuntará a cada píxel.

Ejecute el siguiente código para imprimir la matriz y ver el formato de imagen final en el mapa.

In [ ]:
# formatear en imagen multibanda
coeffsSeg = coeffsSeg.arrayFlatten([ee.List(bns),ee.List(harmonicTag)])
coeffsSeg = timeImg.addBands(coeffsSeg)

# Formatear en un marco de datos para verlo en un punto
df = g2p.extractPointValuesToDataFrame(coeffsSeg,pt,scale=None,crs = crs, transform = transform).transpose()
df.columns = ['Value']

# mostrar
display(df)

# Añadir al mapa
Map.clearMap()
Map.addLayer(coeffsSeg, {}, "coeffsSeg")

Map.turnOnInspector()
Map.view()


,Value
NDVI_coefs_COS1,0.029023
NDVI_coefs_COS2,0.035795
NDVI_coefs_COS3,0.026256
NDVI_coefs_INTP,-183.772740
NDVI_coefs_SIN1,-0.012175
NDVI_coefs_SIN2,-0.006583
NDVI_coefs_SIN3,-0.005892
NDVI_coefs_SLP,0.091429
blue_coefs_COS1,0.000000
blue_coefs_COS2,-0.014370


Adding layer: coeffsSeg
Starting webmap
Using default refresh token for geeView: C:\Users\ianho/.config/earthengine/credentials
Local web server at: http://localhost:1233/geeView/ already serving.
cwd c:\RCR\quickLabsTrainingMaterials\lcms-training


### 3.5.11: Predecir valores del modelo para una fecha determinada

* Ahora hemos aprendido cómo extraer el modelo armónico para una fecha determinada.
* A continuación, aprenderemos cómo predecir valores para el modelo para esa fecha determinada.

La función `changeDetectionLib.simpleCCDCPrediction` se utiliza para obtener los **valores previstos** para una fecha determinada.

Inspeccione la estructura de la función a continuación. Presta atención a:
- a) variables de entrada
- b) preprocesamiento de la imagen y coeficientes
- c) objetos de salida

In [ ]:
# inspeccionar la función
print(inspect.getsource(changeDetectionLib.simpleCCDCPrediction) )

def simpleCCDCPrediction(img,timeBandName,whichHarmonics,whichBands):
  #Unit of each harmonic (1 cycle)
  omega = ee.Number(2.0).multiply(math.pi)

  #Pull out the time band in the yyyy.ff format
  tBand = img.select([timeBandName])
  
  #Pull out the intercepts and slopes
  intercepts = img.select(['.*_INTP'])
  slopes = img.select(['.*_SLP']).multiply(tBand)
  
  #Set up the omega for each harmonic for the given time band
  tOmega = ee.Image(whichHarmonics).multiply(omega).multiply(tBand)
  cosHarm = tOmega.cos()
  sinHarm = tOmega.sin()
  
  #Set up which harmonics to select

  harmSelect = ee.List(whichHarmonics).map(lambda n: ee.String('.*').cat(ee.Number(n).format()))
  
  #Select the harmonics specified
  sins = img.select(['.*_SIN.*'])
  sins = sins.select(harmSelect)
  coss = img.select(['.*_COS.*'])
  coss = coss.select(harmSelect)
  
  #Set up final output band names
  outBns = ee.List(whichBands).map(lambda bn: ee.String(bn).cat('_CCDC_fitted'))
  
  #Iterate across each b

#### Aplicar y predecir el modelo armónico
Ahora veremos cómo aplicar y predecir el modelo armónico para una sola fecha, en función de los coeficientes que extrajimos anteriormente.

Tenga en cuenta que el proceso siguiente es idéntico a la función "simpleCCDCPrediction". Revisará la función `simpleCCDCPrediction` paso a paso.

In [ ]:
# establecer imagen de entrada
img = coeffsSeg

# Especifique qué armónicos aplicar (1,2,3 están disponibles)
whichHarmonics = [1,2,3]

# Encuentre qué bandas están disponibles para predecir
whichBands = coeffsSeg.select(['.*_INTP']).bandNames().map(lambda bn: ee.String(bn).split('_').get(0))
whichBands = ee.Dictionary(whichBands.reduce(ee.Reducer.frequencyHistogram())).keys().getInfo()

#Encuentre qué bandas están disponibles para predecir
omega = ee.Number(2.0).multiply(changeDetectionLib.math.pi)

#Saque la franja horaria en formato aaaa.ff (yyyy.ff) 
tBand = img.select([timeBandName])

#Saca las intersecciones y pendientes.
intercepts = img.select(['.*_INTP'])
slopes = img.select(['.*_SLP']).multiply(tBand)

#Configurar el omega para cada armónico para la franja horaria determinada
tOmega = ee.Image(whichHarmonics).multiply(omega).multiply(tBand)
cosHarm = tOmega.cos()
sinHarm = tOmega.sin()

#Configurar qué armónicos seleccionar
harmSelect = ee.List(whichHarmonics).map(lambda n: ee.String('.*').cat(ee.Number(n).format()))

#Seleccione los armónicos especificados
sins = img.select(['.*_SIN.*'])
sins = sins.select(harmSelect)
coss = img.select(['.*_COS.*'])
coss = coss.select(harmSelect)

#Configurar nombres de bandas de salida final
outBns = ee.List(whichBands).map(lambda bn: ee.String(bn).cat('_CCDC_fitted'))

#Iterar a través de cada banda y predecir el valor
def predHelper(bn):
    bn = ee.String(bn);
    return ee.Image([intercepts.select(bn.cat('_.*')),
                    slopes.select(bn.cat('_.*')),
                    sins.select(bn.cat('_.*')).multiply(sinHarm),
                    coss.select(bn.cat('_.*')).multiply(cosHarm)
                    ]).reduce(ee.Reducer.sum());
predicted = ee.ImageCollection(list(map(predHelper,whichBands))).toBands().rename(outBns)


# Ver los valores previstos en el mapa
Map.clearMap()
Map.addLayer(predicted,{'bands':'swir1_CCDC_fitted,nir_CCDC_fitted,red_CCDC_fitted','min':0.05,'max':[0.5,0.8,0.4]},f'Predicted CCDC {date}')
Map.turnOnInspector()
Map.view()

Adding layer: Predicted CCDC 2017.9
Starting webmap
Using default refresh token for geeView: C:\Users\ianho/.config/earthengine/credentials
Local web server at: http://localhost:1233/geeView/ already serving.
cwd c:\RCR\quickLabsTrainingMaterials\lcms-training


#### Inspeccionar salidas
Haga clic en el mapa para consultar los valores en un punto. También puede ejecutar el bloque de código siguiente para ver el resultado, los valores ajustados en el punto que consultó anteriormente.

Las salidas aquí representan los valores de banda ajustados del modelo CCDC en un único punto de tiempo.

In [ ]:
# Formatear en un marco de datos para verlo en un punto
df = g2p.extractPointValuesToDataFrame(predicted,pt,scale=None,crs = crs, transform = transform).transpose()
df.columns = ['Value']

# mostrar
display(df)

,Value
NDVI_CCDC_fitted,0.767051
blue_CCDC_fitted,0.028508
green_CCDC_fitted,0.064697
nir_CCDC_fitted,0.378252
red_CCDC_fitted,0.043496
swir1_CCDC_fitted,0.201752
swir2_CCDC_fitted,0.091913


### 3.5.12: Ponlo todo junto: ejecuta CCDC en una serie temporal

* Ahora predeciremos la salida del CCDC sobre una serie temporal completa de imágenes de fechas desde el principio hasta el final cada décimo de año.
* Esto nos permitirá hacer clic en el mapa y ver la estacionalidad representada por CCDC para varias bandas/índices.

In [ ]:
# Se sigue el mismo flujo de trabajo que con una salida CCDC sin terminar
Map.clearMap()

#Especifique qué armónicos usar al predecir el modelo CCDC
#CCDC exporta los primeros 3 armónicos (1 ciclo/año, 2 ciclos/año y 3 ciclos/año)
#Si solo desea ver patrones anuales, especifique [1]
#Si desea un ajuste más preciso en el valor predicho, incluya también el segundo o tercer armónico [1,2,3]
whichHarmonics = [1,2,3]

#Si se deben llenar los espacios entre el año final de los segmentos y el año de inicio posterior a la fecha de ruptura
fillGaps = False

#Aplicar el modelo armónico CCDC a lo largo de una serie temporal
#Primero obtenga una serie temporal de imágenes temporales con un paso de tiempo de 0,1 de año
yearImages = changeDetectionLib.getTimeImageCollection(startYear,endYear,startJulian,endJulian,0.1);

# Luego predice los modelos CCDC.
fitted = changeDetectionLib.predictCCDC(ccdcImg,yearImages,fillGaps,whichHarmonics)
Map.addLayer(fitted.select(['.*_fitted']),{'bands':'swir1_CCDC_fitted,nir_CCDC_fitted,red_CCDC_fitted','min':0.05,'max':0.6},'Fitted CCDC',True);


# Visualización de compuestos sintéticos
# Tome bandas compuestas de colores falsos comunes y visualícelas durante el penúltimo año

# Primero obtenga las bandas de las bandas previstas y luego divida el nombre
fittedBns = fitted.select(['.*_fitted']).first().bandNames()
bns = fittedBns.map(lambda bn: ee.String(bn).split('_').get(0))

# Filtrar hasta el penúltimo año y un rango de fechas de verano
syntheticComposites = fitted.select(fittedBns,bns)\
    .filter(ee.Filter.calendarRange(endYear-1,endYear-1,'year'))\
    .filter(ee.Filter.calendarRange(60,80)).first()

# Visualiza la salida como lo harías con un compuesto
Map.addLayer(syntheticComposites,getImagesLib.vizParamsFalse,'Synthetic Composite')


#visualizar imágenes del tiempo
Map.addLayer(yearImages, {'opacity':0}, "year images")
Map.turnOnInspector()
Map.view()



Adding layer: Fitted CCDC
Adding layer: Synthetic Composite
Adding layer: year images
Starting webmap
Using default refresh token for geeView: /home/jupyter/.config/earthengine/credentials
Local web server at: http://localhost:1234/geeView/ already serving.
cwd /home/jupyter/lcms-training
Workbench Proxy URL: https://1307eb830a12e633-dot-us-central1.notebooks.googleusercontent.com/proxy/1234/geeView/?accessToken=None


## Laboratorio 3 Desafío 2:

**Para los usuarios de Qwiklabs**, se evaluará su finalización en la parte "Verificar mi progreso" al final de la práctica 2.

1. Utilice la salida CCDC precocida para predecir el NDVI de CCDC de 2015.9 y 2017.9

    * Utilice la siguiente colección de imágenes CCDC:

    ```pitón
          ccdcImg = ee.ImageCollection(f'{pre_baked_path_root}/lcms-training_module-3_CCDC').mosaic()
      ```
      <br>

    * Crea una colección de imágenes de tiempo para las fechas especificadas.

    
    <br>
      
    * Utilice la siguiente función para predecir valores CCDC:

    ```pitón
          changeDetectionLib.predictCCDC(ccdcImg,timeImgs,fillGaps=True,whatHarmonics=[1,2,3])
      ```
      <br>
   
   

<br>

2. Reste los valores de NDVI pronosticados para 2015.9 de los valores de NDVI pronosticados para 2017.9 para mostrar el cambio (`bandName = 'NDVI_CCDC_fitted'`). Opcionalmente, puede agregar esta capa al mapa para ver el cambio entre 2015 y 2017 cuando el huracán María azotó el área de estudio.


3. Extraiga el valor de diferencia para la siguiente ubicación.

* Utilice un punto con estas coordenadas: `([-65.844, 18.261])`
        
* Utilice la siguiente función: `g2p.extractPointValuesToDataFrame`
    <br>
    Ejemplo:
    ```pitón
            valores_extraídos = g2p.extractPointValuesToDataFrame(
            diferencia,
            ee.Geometry.Point([-65.844, 18.261]),
            escala = 30,
            crs = "EPSG:5070",
            transformar = Ninguno,
            reductor = ee.Reducer.first(),
            incluirNonSystemProperties = Falso,
            incluirSystemProperties=Verdadero
            )
    ```
    <br>
4. Guarde los valores extraídos en un archivo csv.

   * Guarde csv en esta ruta: `"/tmp/challenge/module_3_challenge2_answer.csv"`
     * **Nota: La ruta al csv debe coincidir exactamente con la ruta anterior.**
    <br>
    
    * Cree la carpeta `"/tmp/challenge"` si aún no existe.
      
        Ejemplo:
    ```pitón
        out_csv = "/tmp/challenge/module_3_challenge2_answer.csv"
        si no, os.path.exists(os.path.dirname(out_csv)):os.makedirs(os.path.dirname(out_csv))
    ```
<br>

5. Verifique que exista el csv de salida.
    
    * Ejemplo:
    ```pitón
        imprimir(os.path.exists(out_csv))
    ```
<br>

In [ ]:
# Ponga la solución del código de desafío aquí


## Listo con el Módulo 3

Las predicciones de CCDC y LandTrendr se utilizarán como entradas en módulos posteriores.